## 00. 설정

In [133]:
import os
import random
import collections
from pathlib import Path

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ======================
# 0. 환경 기본 설정
# ======================

# 데이터 루트 경로: 
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")

# 파일 이름 
CT_NAME   = "ct.nii.gz"
MASK_NAME = "whole_artery_cleaned.nii.gz"
   

PATCH_SIZE = (32, 32, 32)   # (D, H, W)

# 디바이스 설정 (M2 맥: mps 사용)
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

print("Using device:", DEVICE)

Using device: mps


## 01. action 을 통해 이동한 voxel 이 정답 혈관(mask.nii) volume 내에 있는지에 따른 보상

In [11]:

import os
import random
import collections
from pathlib import Path

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ======================
# 0. 환경 기본 설정
# ======================

# 데이터 루트 경로: RL_prepared 폴더
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/RL_prepared")

# 파일 이름 (필요하면 여기만 수정)
CT_NAME   = "ct.nii.gz"
MASK_NAME = "Whole_artery.nii.gz"   # 폴더 안 파일명 확인해서 맞추기

PATCH_SIZE = (32, 32, 32)   # (D, H, W)

# 디바이스 설정 (M2 맥: mps 사용)
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

print("Using device:", DEVICE)

# ======================
# 1. 데이터 로딩 & 전처리
# ======================

def load_case(case_dir: Path,
              ct_name: str = CT_NAME,
              mask_name: str = MASK_NAME):
    """
    case_dir: .../RL_prepared/01011ug_500 이런 폴더
    안에 ct.nii.gz, Whole_artery.nii.gz 가 있어야 함
    """
    ct_path   = case_dir / ct_name
    mask_path = case_dir / mask_name

    if not ct_path.exists():
        raise FileNotFoundError(f"CT not found: {ct_path}")
    if not mask_path.exists():
        raise FileNotFoundError(f"Mask not found: {mask_path}")

    ct_nii   = nib.load(str(ct_path))
    mask_nii = nib.load(str(mask_path))

    ct   = ct_nii.get_fdata().astype(np.float32)    # (D,H,W)
    mask = mask_nii.get_fdata().astype(np.float32)  # (D,H,W), 0/1 또는 label

    if ct.shape != mask.shape:
        raise ValueError(f"Shape mismatch: CT {ct.shape}, MASK {mask.shape} at {case_dir}")

    return ct, mask

def normalize_ct(ct: np.ndarray,
                 hu_min: float = -200.0,
                 hu_max: float = 300.0) -> np.ndarray:
    """
    CT HU 값을 [hu_min, hu_max]로 클리핑 후 [0,1]로 정규화
    """
    ct = np.clip(ct, hu_min, hu_max)
    ct = (ct - hu_min) / (hu_max - hu_min + 1e-8)
    return ct.astype(np.float32)

def get_patch(volume: np.ndarray,
              center: tuple,
              patch_size=PATCH_SIZE) -> np.ndarray:
    """
    volume: (D,H,W)
    center: (z,y,x)
    patch_size: (pd,ph,pw)
    - 볼륨 밖으로 나가는 경우 0 padding
    """
    D, H, W = volume.shape
    pd, ph, pw = patch_size
    zc, yc, xc = center

    z0 = zc - pd // 2
    y0 = yc - ph // 2
    x0 = xc - pw // 2
    z1 = z0 + pd
    y1 = y0 + ph
    x1 = x0 + pw

    patch = np.zeros((pd, ph, pw), dtype=volume.dtype)

    z0_src = max(z0, 0)
    y0_src = max(y0, 0)
    x0_src = max(x0, 0)
    z1_src = min(z1, D)
    y1_src = min(y1, H)
    x1_src = min(x1, W)

    z0_dst = z0_src - z0
    y0_dst = y0_src - y0
    x0_dst = x0_src - x0

    patch[
        z0_dst:z0_dst + (z1_src - z0_src),
        y0_dst:y0_dst + (y1_src - y0_src),
        x0_dst:x0_dst + (x1_src - x0_src)
    ] = volume[z0_src:z1_src, y0_src:y1_src, x0_src:x1_src]

    return patch


# ======================
# 2. CT Vessel Env 정의
# ======================

class CTVesselEnv:
    """
    - 상태: 현재 위치 주변의 32x32x32 CT 패치 (normalize됨)
    - 행동: 6방향 이동 (±z, ±y, ±x)
    - 보상:
        * 새로 이동한 voxel 이 혈관(mask>0)이면 +1
        * 혈관이 아니면 -1, 그리고 에피소드 종료(done=True)
    """

    def __init__(self,
                 data_root: Path,
                 patch_size=PATCH_SIZE,
                 max_steps: int = 32):
        self.data_root = Path(data_root)
        self.patch_size = patch_size
        self.max_steps = max_steps

        # case 폴더 모으기
        self.case_dirs = sorted(
            [p for p in self.data_root.iterdir() if p.is_dir()]
        )
        if len(self.case_dirs) == 0:
            raise RuntimeError(f"No case folders found under {self.data_root}")

        print(f"Found {len(self.case_dirs)} cases under {self.data_root}")

        # 내부 상태
        self.ct = None
        self.mask = None
        self.ct_norm = None
        self.D = self.H = self.W = None

        self.pos = None   # (z,y,x)
        self.step_count = 0
        self.current_case_dir = None

    def _load_random_case(self):
        """
        케이스 하나 랜덤 선택 후 CT, mask 로딩 + 정규화
        """
        while True:
            case_dir = random.choice(self.case_dirs)
            try:
                ct, mask = load_case(case_dir)
            except Exception as e:
                print(f"[WARN] skip case {case_dir.name}: {e}")
                continue

            # vessel 존재 여부 확인
            if (mask > 0).sum() == 0:
                print(f"[WARN] case {case_dir.name} has no vessel voxels, skip")
                continue

            self.ct = ct
            self.mask = (mask > 0.5).astype(np.uint8)  # binary
            self.ct_norm = normalize_ct(ct)

            self.D, self.H, self.W = self.ct.shape
            self.current_case_dir = case_dir
            break

    def _sample_start_pos(self):
        """
        혈관(mask>0) 안의 voxel 중 하나 랜덤 선택
        """
        vessel_indices = np.argwhere(self.mask > 0)  # (N,3)
        idx = vessel_indices[np.random.randint(len(vessel_indices))]
        return tuple(idx)  # (z,y,x)

    def _get_state(self):
        """
        현재 위치 self.pos 기준 패치 -> (1, D, H, W)
        """
        patch = get_patch(self.ct_norm, self.pos, self.patch_size)
        state = patch[np.newaxis, ...].astype(np.float32)
        return state

    def _move(self, z, y, x, action: int):
        """
        action: 0~5
          0: +z
          1: -z
          2: +y
          3: -y
          4: +x
          5: -x
        """
        if action == 0:
            z += 1
        elif action == 1:
            z -= 1
        elif action == 2:
            y += 1
        elif action == 3:
            y -= 1
        elif action == 4:
            x += 1
        elif action == 5:
            x -= 1
        else:
            raise ValueError(f"Invalid action: {action}")
        return z, y, x

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0

        state = self._get_state()
        info = {
            "pos": self.pos,
            "case": self.current_case_dir.name,
        }
        return state, info

    def step(self, action: int):
        """
        핵심 Reward 설계:
        - 새 위치가 볼륨 밖이면: reward=-1, done=True
        - 볼륨 안이지만 혈관이 아니면: reward=-1, done=True
        - 혈관이면: reward=+1, done=False (단, max_steps 도달 시 done=True)
        """
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)

        # 볼륨 바깥 체크
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            reward = -1.0
            done = True
            obs = self._get_state()  # 어차피 done 이라 크게 의미 없음
            info = {
                "pos": (nz, ny, nx),
                "inside": False,
                "case": self.current_case_dir.name,
            }
            return obs, reward, done, False, info

        # 혈관 내부 여부
        inside = bool(self.mask[nz, ny, nx] > 0)

        if inside:
            reward = +1.0
            done = False
        else:
            reward = -1.0
            done = True   # 혈관 벗어나면 바로 종료

        # 상태 업데이트
        self.pos = (nz, ny, nx)
        self.step_count += 1

        if self.step_count >= self.max_steps:
            done = True

        obs = self._get_state()
        info = {
            "pos": self.pos,
            "inside": inside,
            "case": self.current_case_dir.name,
        }
        return obs, reward, done, False, info


# ======================
# 3. 3D CNN Q-network
# ======================

class CnnQNet(nn.Module):
    """
    입력:  (B, 1, 32, 32, 32)
    출력:  (B, n_actions)
    """
    def __init__(self, in_channels=1, n_actions=6):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, 8, 3, padding=1)
        self.bn1   = nn.BatchNorm3d(8)
        self.conv2 = nn.Conv3d(8, 16, 3, padding=1)
        self.bn2   = nn.BatchNorm3d(16)
        self.conv3 = nn.Conv3d(16, 32, 3, padding=1)
        self.bn3   = nn.BatchNorm3d(32)

        self.pool  = nn.MaxPool3d(kernel_size=2, stride=2)

        # 32x32x32  →  16x16x16 →  8x8x8 → 4x4x4, 채널 32
        self.fc1   = nn.Linear(32 * 4 * 4 * 4, 128)
        self.fc2   = nn.Linear(128, n_actions)

    def forward(self, x):
        # x: (B,1,32,32,32)
        x = self.pool(F.relu(self.bn1(self.conv1(x))))   # (B,8,16,16,16)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))   # (B,16,8,8,8)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))   # (B,32,4,4,4)
        x = torch.flatten(x, 1)                          # (B, 32*4*4*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                                  # (B, n_actions)
        return x

    def sample_action(self, obs_np: np.ndarray, epsilon: float):
        """
        obs_np: numpy (1, 32, 32, 32)
        """
        if random.random() < epsilon:
            return random.randint(0, self.fc2.out_features - 1)
        # greedy
        obs_t = torch.from_numpy(obs_np).float().unsqueeze(0).to(DEVICE)  # (1,1,32,32,32)
        with torch.no_grad():
            q_values = self.forward(obs_t)
            action = q_values.argmax(dim=1).item()
        return action


# ======================
# 4. Replay Buffer & DQN 학습
# ======================

class ReplayBuffer:
    def __init__(self, buffer_limit=20000):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        # transition: (s, a, r, s_prime, done_mask)
        self.buffer.append(transition)

    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        for (s, a, r, s_prime, done_mask) in mini_batch:
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        s       = torch.tensor(s_lst, dtype=torch.float32, device=DEVICE)      # (B,1,32,32,32)
        a       = torch.tensor(a_lst, device=DEVICE)                           # (B,1)
        r       = torch.tensor(r_lst, dtype=torch.float32, device=DEVICE)      # (B,1)
        s_prime = torch.tensor(s_prime_lst, dtype=torch.float32, device=DEVICE)# (B,1,32,32,32)
        done_m  = torch.tensor(done_mask_lst, dtype=torch.float32, device=DEVICE) # (B,1)

        return s, a, r, s_prime, done_m

    def size(self):
        return len(self.buffer)


def train_dqn(q_net, target_net, memory, optimizer,
              batch_size=8, gamma=0.99, iters=1):
    if memory.size() < batch_size:
        return

    for _ in range(iters):
        s, a, r, s_prime, done_m = memory.sample(batch_size)

        q_out = q_net(s)                 # (B, n_actions)
        q_a   = q_out.gather(1, a)       # (B,1)

        with torch.no_grad():
            max_q_prime = target_net(s_prime).max(1, keepdim=True)[0]  # (B,1)
            target = r + gamma * max_q_prime * done_m

        loss = F.mse_loss(q_a, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


# ======================
# 5. 학습 루프
# ======================

def train_vessel_dqn(n_episodes=200,
                     max_steps=32,
                     buffer_limit=20000,
                     batch_size=8):
    env = CTVesselEnv(DATA_ROOT, patch_size=PATCH_SIZE, max_steps=max_steps)

    q_net      = CnnQNet(in_channels=1, n_actions=6).to(DEVICE)
    target_net = CnnQNet(in_channels=1, n_actions=6).to(DEVICE)
    target_net.load_state_dict(q_net.state_dict())

    memory = ReplayBuffer(buffer_limit=buffer_limit)
    optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

    print_interval = 10
    global_step = 0

    for ep in range(1, n_episodes + 1):
        state, info = env.reset()
        ep_reward = 0.0
        ep_steps = 0

        # epsilon 선형 감소
        epsilon = max(0.01, 0.1 - 0.01 * (ep / 50.0))

        for t in range(max_steps):
            action = q_net.sample_action(state, epsilon)
            next_state, reward, done, truncated, info2 = env.step(action)

            done_mask = 0.0 if (not done) else 1.0

            memory.put((state, action, reward, next_state, done_mask))

            state = next_state
            ep_reward += reward
            ep_steps += 1
            global_step += 1

            # 학습
            if memory.size() > 200:
                train_dqn(q_net, target_net, memory, optimizer,
                          batch_size=batch_size, gamma=0.99, iters=1)

            if done:
                break

        # 타겟 네트워크 주기적으로 동기화
        if ep % 20 == 0:
            target_net.load_state_dict(q_net.state_dict())

        if ep % print_interval == 0:
            print(f"[Episode {ep:03d}] "
                  f"reward={ep_reward:.1f}, "
                  f"steps={ep_steps}, "
                  f"eps={epsilon:.3f}, "
                  f"buffer={memory.size()}, "
                  f"case={info2.get('case', 'NA')}, "
                  f"inside={info2.get('inside', False)}")

    print("Training finished.")
    return q_net, target_net, env


# ======================
# 6. 실행
# ======================

q_net, target_net, env = train_vessel_dqn(
    n_episodes=50,   # 처음엔 50 정도만 돌려보고
    max_steps=32,
    buffer_limit=5000,
    batch_size=8
)

Using device: mps
Found 100 cases under /Users/srpark/Projects/RL_project/data/RL_prepared
[Episode 010] reward=6.0, steps=8, eps=0.098, buffer=121, case=01015ug_91, inside=False


/var/folders/78/kff0l0ts7cnc7n54tdllr0qc0000gn/T/ipykernel_49145/267094466.py:358: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  s       = torch.tensor(s_lst, dtype=torch.float32, device=DEVICE)      # (B,1,32,32,32)


[Episode 020] reward=-1.0, steps=1, eps=0.096, buffer=241, case=01011ug_528, inside=False
[Episode 030] reward=11.0, steps=13, eps=0.094, buffer=430, case=01011ug_506, inside=False
[Episode 040] reward=32.0, steps=32, eps=0.092, buffer=563, case=01010ug_136, inside=True
[Episode 050] reward=32.0, steps=32, eps=0.090, buffer=715, case=01018ug_88, inside=True
Training finished.


In [12]:
class CTVesselEnv:
    def __init__(self, data_root: Path, patch_size=PATCH_SIZE, max_steps=32):
        # ... (기존 초기화 코드 동일) ...
        self.visited = set()  # 방문 좌표 기록용 집합 추가

    def reset(self, seed=None):
        # ... (기존 로직 동일) ...
        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0
        
        # 방문 기록 초기화 및 시작점 추가
        self.visited = set()
        self.visited.add(self.pos) 

        state = self._get_state()
        info = {"pos": self.pos, "case": self.current_case_dir.name}
        return state, info

    def step(self, action: int):
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)

        # 1. 볼륨 밖으로 나감
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            return self._get_state(), -1.0, True, False, {"inside": False}

        # 2. 혈관 여부 확인
        inside = bool(self.mask[nz, ny, nx] > 0)
        
        if not inside:
            # 혈관 밖으로 나감 -> 페널티 받고 종료
            reward = -1.0
            done = True
        else:
            # 혈관 안쪽임
            if (nz, ny, nx) in self.visited:
                # 이미 방문했던 곳 -> 페널티를 조금 주거나 0점 처리 (진동 방지)
                reward = -0.1 
                done = False # 죽이진 않음
            else:
                # 새로운 혈관 발견! -> 보상
                reward = +1.0
                done = False
                self.visited.add((nz, ny, nx))

        # 위치 업데이트 (혈관 밖이라도 이동은 함, 어차피 done 되지만 시각화 등을 위해)
        self.pos = (nz, ny, nx)
        self.step_count += 1

        if self.step_count >= self.max_steps:
            done = True

        return self._get_state(), reward, done, False, {"inside": inside}

In [14]:
import os
import random
import collections
import math
from pathlib import Path

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ======================
# 0. 환경 기본 설정
# ======================

# [수정 필요] 데이터 경로
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/RL_prepared")

CT_NAME   = "ct.nii.gz"
MASK_NAME = "Whole_artery.nii.gz"

PATCH_SIZE = (32, 32, 32)  # (D, H, W)

# 디바이스 설정
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print("Using device:", DEVICE)


# ======================
# 1. 데이터 로딩 & 전처리 함수
# ======================

def load_case(case_dir: Path, ct_name: str = CT_NAME, mask_name: str = MASK_NAME):
    ct_path   = case_dir / ct_name
    mask_path = case_dir / mask_name

    if not ct_path.exists(): raise FileNotFoundError(f"CT not found: {ct_path}")
    if not mask_path.exists(): raise FileNotFoundError(f"Mask not found: {mask_path}")

    ct_nii   = nib.load(str(ct_path))
    mask_nii = nib.load(str(mask_path))

    ct   = ct_nii.get_fdata().astype(np.float32)
    mask = mask_nii.get_fdata().astype(np.float32)

    return ct, mask

def normalize_ct(ct: np.ndarray, hu_min=-200.0, hu_max=300.0):
    ct = np.clip(ct, hu_min, hu_max)
    ct = (ct - hu_min) / (hu_max - hu_min + 1e-8)
    return ct.astype(np.float32)

def get_patch(volume: np.ndarray, center: tuple, patch_size=PATCH_SIZE):
    D, H, W = volume.shape
    pd, ph, pw = patch_size
    zc, yc, xc = center

    z0, y0, x0 = zc - pd // 2, yc - ph // 2, xc - pw // 2
    z1, y1, x1 = z0 + pd, y0 + ph, x0 + pw

    patch = np.zeros((pd, ph, pw), dtype=volume.dtype)

    z0_src, y0_src, x0_src = max(z0, 0), max(y0, 0), max(x0, 0)
    z1_src, y1_src, x1_src = min(z1, D), min(y1, H), min(x1, W)

    z0_dst, y0_dst, x0_dst = z0_src - z0, y0_src - y0, x0_src - x0

    # 범위가 유효할 때만 복사
    if z1_src > z0_src and y1_src > y0_src and x1_src > x0_src:
        patch[
            z0_dst:z0_dst + (z1_src - z0_src),
            y0_dst:y0_dst + (y1_src - y0_src),
            x0_dst:x0_dst + (x1_src - x0_src)
        ] = volume[z0_src:z1_src, y0_src:y1_src, x0_src:x1_src]

    return patch


# ======================
# 2. CT Vessel Env 정의 (개선됨)
# ======================

class CTVesselEnv:
    """
    [개선 사항]
    1. Observation: 2채널 (CT 이미지 + 방문한 경로 Mask)
    2. Reward: 
       - 새로운 혈관 발견: +1.0
       - 이미 갔던 혈관: -0.1 (제자리 진동 방지)
       - 혈관 밖/이탈: -1.0
    """
    def __init__(self, data_root: Path, patch_size=PATCH_SIZE, max_steps=100):
        self.data_root = Path(data_root)
        self.patch_size = patch_size
        self.max_steps = max_steps  # 스텝 수 좀 늘려줌

        self.case_dirs = sorted([p for p in self.data_root.iterdir() if p.is_dir()])
        if not self.case_dirs:
            raise RuntimeError(f"No case folders found under {self.data_root}")
        print(f"Found {len(self.case_dirs)} cases.")

        # 내부 변수
        self.ct = None
        self.mask = None
        self.ct_norm = None
        self.visited_vol = None  # 관측(Observation) 생성용 3D 볼륨
        self.visited_set = None  # 보상 계산용 빠른 조회 Set
        
        self.D = self.H = self.W = None
        self.pos = None
        self.step_count = 0
        self.current_case_dir = None

    def _load_random_case(self):
        while True:
            case_dir = random.choice(self.case_dirs)
            try:
                ct, mask = load_case(case_dir)
            except Exception:
                continue

            if (mask > 0).sum() == 0: continue

            self.ct = ct
            self.mask = (mask > 0.5).astype(np.uint8)
            self.ct_norm = normalize_ct(ct)
            
            # [중요] 방문 기록용 볼륨 초기화 (0으로 채움)
            self.visited_vol = np.zeros_like(self.mask, dtype=np.float32)
            
            self.D, self.H, self.W = self.ct.shape
            self.current_case_dir = case_dir
            break

    def _sample_start_pos(self):
        vessel_indices = np.argwhere(self.mask > 0)
        idx = vessel_indices[np.random.randint(len(vessel_indices))]
        return tuple(idx)

    def _get_state(self):
        """
        [수정] 2채널 입력 생성: (2, D, H, W)
        Channel 0: CT Patch
        Channel 1: Visited Mask Patch
        """
        # 1. CT 패치
        ct_patch = get_patch(self.ct_norm, self.pos, self.patch_size)
        # 2. 방문 기록 패치
        vis_patch = get_patch(self.visited_vol, self.pos, self.patch_size)
        
        # 두 패치를 합침 -> (2, 32, 32, 32)
        state = np.stack([ct_patch, vis_patch], axis=0).astype(np.float32)
        return state

    def _move(self, z, y, x, action):
        # 0:+z, 1:-z, 2:+y, 3:-y, 4:+x, 5:-x
        deltas = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]
        dz, dy, dx = deltas[action]
        return z+dz, y+dy, x+dx

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0
        
        # [중요] 방문 기록 초기화
        self.visited_set = set()
        self.visited_set.add(self.pos)
        self.visited_vol[self.pos] = 1.0  # 볼륨에도 표시

        state = self._get_state()
        info = {"pos": self.pos, "case": self.current_case_dir.name}
        return state, info

    def step(self, action: int):
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)
        
        # 1. 볼륨 밖 이탈 체크
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            reward = -1.0
            done = True
            return self._get_state(), reward, done, False, {"inside": False}

        # 2. 혈관 내부 여부 체크
        inside = bool(self.mask[nz, ny, nx] > 0)

        if not inside:
            # 혈관 밖 -> 페널티 받고 종료
            reward = -1.0
            done = True
        else:
            # 혈관 안쪽
            if (nz, ny, nx) in self.visited_set:
                # [중요] 이미 갔던 곳 -> 약한 페널티 (제자리 걸음 방지)
                reward = -0.1
                done = False
            else:
                # [중요] 새로운 혈관 -> 보상 (+1)
                reward = 1.0
                done = False
                # 방문 기록 업데이트
                self.visited_set.add((nz, ny, nx))
                self.visited_vol[nz, ny, nx] = 1.0

        self.pos = (nz, ny, nx)
        self.step_count += 1
        
        if self.step_count >= self.max_steps:
            done = True

        return self._get_state(), reward, done, False, {"inside": inside}


# ======================
# 3. 3D CNN Q-network (2채널 입력 대응)
# ======================

class CnnQNet(nn.Module):
    def __init__(self, in_channels=2, n_actions=6): # [수정] 기본 채널 2
        super().__init__()
        # BatchNorm 제거 (안정적인 학습을 위해)
        self.conv1 = nn.Conv3d(in_channels, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)
        
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # (B, 64, 4, 4, 4) -> 64*4*4*4 = 4096
        self.fc1 = nn.Linear(64 * 4 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, n_actions)

    def forward(self, x):
        # x: (B, 2, 32, 32, 32)
        x = self.pool(F.relu(self.conv1(x))) # -> (16, 16, 16, 16)
        x = self.pool(F.relu(self.conv2(x))) # -> (32, 8, 8, 8)
        x = self.pool(F.relu(self.conv3(x))) # -> (64, 4, 4, 4)

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def sample_action(self, obs_np, epsilon):
        if random.random() < epsilon:
            return random.randint(0, self.fc2.out_features - 1)
        
        # obs_np: (2, 32, 32, 32)
        obs_t = torch.from_numpy(obs_np).float().unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            q_out = self.forward(obs_t)
            action = q_out.argmax(dim=1).item()
        return action


# ======================
# 4. Replay Buffer & Train Function
# ======================

class ReplayBuffer:
    def __init__(self, buffer_limit=50000):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for (s, a, r, s_prime, done_mask) in mini_batch:
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return (torch.tensor(np.array(s_lst), dtype=torch.float32, device=DEVICE),
                torch.tensor(a_lst, dtype=torch.int64, device=DEVICE),
                torch.tensor(r_lst, dtype=torch.float32, device=DEVICE),
                torch.tensor(np.array(s_prime_lst), dtype=torch.float32, device=DEVICE),
                torch.tensor(done_mask_lst, dtype=torch.float32, device=DEVICE))

    def size(self):
        return len(self.buffer)

def train_dqn(q_net, target_net, memory, optimizer, batch_size=32, gamma=0.99):
    if memory.size() < batch_size: return

    s, a, r, s_prime, done_mask = memory.sample(batch_size)

    q_out = q_net(s)
    q_a = q_out.gather(1, a)

    with torch.no_grad():
        max_q_prime = target_net(s_prime).max(1, keepdim=True)[0]
        # [수정] Done일 때(0.0)는 미래가치 없음
        target = r + gamma * max_q_prime * done_mask

    # Huber Loss (SmoothL1) 추천
    loss = F.smooth_l1_loss(q_a, target)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# ======================
# 5. Main Training Loop
# ======================

def train_vessel_dqn(n_episodes=500, max_steps=128, buffer_limit=50000, batch_size=32):
    # [설정] max_steps를 32에서 128로 늘림 (혈관을 더 길게 따라가도록)
    env = CTVesselEnv(DATA_ROOT, patch_size=PATCH_SIZE, max_steps=max_steps)

    # [설정] 입력 채널 2 (CT, Visited)
    q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net.load_state_dict(q_net.state_dict())

    memory = ReplayBuffer(buffer_limit)
    optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

    print_interval = 10
    
    for ep in range(1, n_episodes + 1):
        state, info = env.reset()
        ep_reward = 0.0
        ep_steps = 0
        
        # [수정] Epsilon Decay: 1.0(완전랜덤) -> 0.01(거의 최적) 
        # 처음 50% 에피소드 동안 선형 감소
        epsilon = max(0.01, 1.0 - (ep / (n_episodes * 0.5)))

        for t in range(max_steps):
            action = q_net.sample_action(state, epsilon)
            next_state, reward, done, _, info2 = env.step(action)

            # [수정] Done Mask: 종료되면 0, 아니면 1
            done_mask = 0.0 if done else 1.0

            memory.put((state, action, reward, next_state, done_mask))

            state = next_state
            ep_reward += reward
            ep_steps += 1

            # 학습 수행
            train_dqn(q_net, target_net, memory, optimizer, batch_size)

            if done: break
        
        # 타겟 네트워크 동기화
        if ep % 20 == 0:
            target_net.load_state_dict(q_net.state_dict())

        if ep % print_interval == 0:
            print(f"[Ep {ep:03d}] Score: {ep_reward:6.1f} | Steps: {ep_steps:3d} | "
                  f"Eps: {epsilon:.2f} | Case: {info2.get('case')} | Inside: {info2.get('inside')}")

    print("Training Finished!")
    return q_net, target_net, env

# ======================
# 6. 실행
# ======================
if __name__ == "__main__":
    q_net, target_net, env = train_vessel_dqn(
        n_episodes=300,   # 300판 정도 추천
        max_steps=128,    # 스텝 수 충분히
        buffer_limit=50000,
        batch_size=32
    )

Using device: mps
Found 100 cases.
[Ep 010] Score:   -1.0 | Steps:   1 | Eps: 0.93 | Case: None | Inside: False
[Ep 020] Score:   54.2 | Steps:  76 | Eps: 0.87 | Case: None | Inside: False
[Ep 030] Score:   91.5 | Steps: 121 | Eps: 0.80 | Case: None | Inside: False
[Ep 040] Score:   14.8 | Steps:  19 | Eps: 0.73 | Case: None | Inside: False
[Ep 050] Score:    3.0 | Steps:   5 | Eps: 0.67 | Case: None | Inside: False
[Ep 060] Score:   25.6 | Steps:  32 | Eps: 0.60 | Case: None | Inside: False
[Ep 070] Score:   91.3 | Steps: 101 | Eps: 0.53 | Case: None | Inside: False
[Ep 080] Score:   -1.0 | Steps:   1 | Eps: 0.47 | Case: None | Inside: False
[Ep 090] Score:  110.0 | Steps: 123 | Eps: 0.40 | Case: None | Inside: False
[Ep 100] Score:   26.4 | Steps:  35 | Eps: 0.33 | Case: None | Inside: False
[Ep 110] Score:  119.2 | Steps: 128 | Eps: 0.27 | Case: None | Inside: True
[Ep 120] Score:  125.8 | Steps: 128 | Eps: 0.20 | Case: None | Inside: True
[Ep 130] Score:  118.1 | Steps: 128 | Eps: 

In [15]:
# 1. 모델 가중치(Weight) 저장
save_path = "vessel_dqn_final.pth"
torch.save(q_net.state_dict(), save_path)
print(f"✅ 모델 저장 완료: {save_path}")
print("이제 커널이 꺼져도 이 파일만 있으면 언제든 다시 불러올 수 있습니다.")

✅ 모델 저장 완료: vessel_dqn_final.pth
이제 커널이 꺼져도 이 파일만 있으면 언제든 다시 불러올 수 있습니다.


In [16]:
def run_inference_and_save(env, q_net, save_name="result_vis.nii.gz"):
    """
    학습된 모델로 1 에피소드를 수행하고, 
    이동 경로(Visited)와 정답(Mask)을 NIfTI 파일로 저장하는 함수
    """
    print(f"\n--- [Test Inference] 시작: {save_name} ---")
    
    # 1. 환경 초기화
    state, info = env.reset()
    case_name = info['case']
    print(f"Case: {case_name} 에서 테스트 진행 중...")

    done = False
    total_reward = 0
    steps = 0
    
    # 2. 에피소드 진행 (탐험 없음, Epsilon=0.0)
    while not done:
        # 무조건 Q값이 가장 높은 행동만 선택 (Greedy)
        action = q_net.sample_action(state, epsilon=0.0)
        
        next_state, reward, done, _, info = env.step(action)
        state = next_state
        total_reward += reward
        steps += 1
        
    print(f"종료! Score: {total_reward:.1f}, Steps: {steps}")

    # 3. 결과 저장 (NIfTI)
    save_dir = Path("inference_results")
    save_dir.mkdir(exist_ok=True)

    # (1) 에이전트가 지나간 경로 (Visited Volume)
    # env.visited_vol 은 0.0 또는 1.0 으로 되어있음
    visited_img = nib.Nifti1Image(env.visited_vol, affine=np.eye(4))
    vis_path = save_dir / f"{case_name}_visited.nii.gz"
    nib.save(visited_img, vis_path)

    # (2) 실제 정답 (Ground Truth Mask) - 비교용
    target_img = nib.Nifti1Image(env.mask, affine=np.eye(4))
    mask_path = save_dir / f"{case_name}_target.nii.gz"
    nib.save(target_img, mask_path)
    
    # (3) 원본 CT (배경용) - 비교용
    ct_img = nib.Nifti1Image(env.ct, affine=np.eye(4))
    ct_path = save_dir / f"{case_name}_ct.nii.gz"
    nib.save(ct_img, ct_path)

    print(f"💾 저장 완료!")
    print(f"   - 경로: {vis_path}")
    print(f"   - 정답: {mask_path}")
    print(f"   - 원본: {ct_path}")
    print("👉 ITK-SNAP 프로그램을 켜서 위 파일들을 드래그해서 열어보세요.")

# 함수 실행 (지금 메모리에 있는 q_net 사용)
run_inference_and_save(env, q_net)


--- [Test Inference] 시작: result_vis.nii.gz ---
Case: 01011ug_527 에서 테스트 진행 중...
종료! Score: 126.9, Steps: 128
💾 저장 완료!
   - 경로: inference_results/01011ug_527_visited.nii.gz
   - 정답: inference_results/01011ug_527_target.nii.gz
   - 원본: inference_results/01011ug_527_ct.nii.gz
👉 ITK-SNAP 프로그램을 켜서 위 파일들을 드래그해서 열어보세요.


aorta 내에서만 칠해지는 문제
skeleton 처럼 그려져서, 추후 dice 계산 불가되는 문제


## 02. Env 에 adaptive brush 도입

1. Brush (붓) 기능: 에이전트가 지나가면 주변(5x5x5)을 칠해서 혈관(Volume)을 채웁니다. (Skeleton 문제 해결)
2. 스마트 보상: 이미 칠해진 곳은 점수를 안 줘서 스스로 새로운 가지(Branch)로 이동하게 합니다. (Aorta 고립 문제 해결)
3. 2채널 입력: CT 원본과 방문 기록(Visited)을 동시에 봐서 길을 판단합니다.
4. 안전한 학습: Epsilon 탐험, Huber Loss, Model Save, Inference 시각화까지 포함했습니다

In [17]:
import os
import random
import collections
import time
from pathlib import Path

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ==========================================
# 0. 설정 (Configuration)
# ==========================================

# [수정 필요] 데이터가 있는 폴더 경로
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/RL_prepared")

CT_NAME   = "ct.nii.gz"
MASK_NAME = "Whole_artery.nii.gz"

# 하이퍼파라미터
PATCH_SIZE = (32, 32, 32)
MAX_STEPS  = 1000        # 혈관 끝까지 가기 위해 충분히 길게 설정
BRUSH_RADIUS = 2         # 붓 크기 (반경 2 = 5x5x5 영역)

BATCH_SIZE = 32
LR         = 1e-4
GAMMA      = 0.99
BUFFER_LIMIT = 50000

# 디바이스 설정
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Using device: {DEVICE}")


# ==========================================
# 1. 데이터 로딩 및 전처리 유틸리티
# ==========================================

def load_case(case_dir: Path, ct_name: str = CT_NAME, mask_name: str = MASK_NAME):
    ct_path   = case_dir / ct_name
    mask_path = case_dir / mask_name

    if not ct_path.exists(): raise FileNotFoundError(f"CT not found: {ct_path}")
    if not mask_path.exists(): raise FileNotFoundError(f"Mask not found: {mask_path}")

    ct_nii   = nib.load(str(ct_path))
    mask_nii = nib.load(str(mask_path))

    ct   = ct_nii.get_fdata().astype(np.float32)
    mask = mask_nii.get_fdata().astype(np.float32)

    return ct, mask

def normalize_ct(ct: np.ndarray, hu_min=-200.0, hu_max=300.0):
    ct = np.clip(ct, hu_min, hu_max)
    ct = (ct - hu_min) / (hu_max - hu_min + 1e-8)
    return ct.astype(np.float32)

def get_patch(volume: np.ndarray, center: tuple, patch_size=PATCH_SIZE):
    D, H, W = volume.shape
    pd, ph, pw = patch_size
    zc, yc, xc = center

    z0, y0, x0 = zc - pd // 2, yc - ph // 2, xc - pw // 2
    z1, y1, x1 = z0 + pd, y0 + ph, x0 + pw

    patch = np.zeros((pd, ph, pw), dtype=volume.dtype)

    z0_src, y0_src, x0_src = max(z0, 0), max(y0, 0), max(x0, 0)
    z1_src, y1_src, x1_src = min(z1, D), min(y1, H), min(x1, W)

    z0_dst, y0_dst, x0_dst = z0_src - z0, y0_src - y0, x0_src - x0

    if z1_src > z0_src and y1_src > y0_src and x1_src > x0_src:
        patch[
            z0_dst:z0_dst + (z1_src - z0_src),
            y0_dst:y0_dst + (y1_src - y0_src),
            x0_dst:x0_dst + (x1_src - x0_src)
        ] = volume[z0_src:z1_src, y0_src:y1_src, x0_src:x1_src]

    return patch


# ==========================================
# 2. 강화학습 환경 (Environment)
# ==========================================

class CTVesselEnv:
    def __init__(self, data_root: Path, patch_size=PATCH_SIZE, max_steps=MAX_STEPS):
        self.data_root = Path(data_root)
        self.patch_size = patch_size
        self.max_steps = max_steps
        self.brush_radius = BRUSH_RADIUS  # 붓 크기 설정

        self.case_dirs = sorted([p for p in self.data_root.iterdir() if p.is_dir()])
        if not self.case_dirs:
            raise RuntimeError(f"No case folders found under {self.data_root}")
        print(f"Found {len(self.case_dirs)} cases.")

        self.ct = None
        self.mask = None
        self.ct_norm = None
        
        # 3D Volume 상태
        self.visited_vol = None 
        
        self.D = self.H = self.W = None
        self.pos = None
        self.step_count = 0
        self.current_case_dir = None

    def _load_random_case(self):
        while True:
            case_dir = random.choice(self.case_dirs)
            try:
                ct, mask = load_case(case_dir)
            except Exception:
                continue
            
            # 혈관이 없는 데이터는 스킵
            if (mask > 0).sum() == 0: continue

            self.ct = ct
            self.mask = (mask > 0.5).astype(np.uint8)
            self.ct_norm = normalize_ct(ct)
            
            # 방문 기록용 빈 볼륨 생성
            self.visited_vol = np.zeros_like(self.mask, dtype=np.float32)
            
            self.D, self.H, self.W = self.ct.shape
            self.current_case_dir = case_dir
            break

    def _sample_start_pos(self):
        # 혈관 내부 랜덤 시작
        vessel_indices = np.argwhere(self.mask > 0)
        idx = vessel_indices[np.random.randint(len(vessel_indices))]
        return tuple(idx)

    def _get_state(self):
        """
        [2채널 입력]
        Ch 0: CT Image
        Ch 1: Visited Mask (내가 칠한 곳)
        """
        ct_patch = get_patch(self.ct_norm, self.pos, self.patch_size)
        vis_patch = get_patch(self.visited_vol, self.pos, self.patch_size)
        state = np.stack([ct_patch, vis_patch], axis=0).astype(np.float32)
        return state

    def _move(self, z, y, x, action):
        # 6방향 이동
        deltas = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]
        dz, dy, dx = deltas[action]
        return z+dz, y+dy, x+dx

    def _apply_brush(self, center_pos):
        """
        [Smart Brush Logic]
        붓 크기만큼 칠하되, '실제 혈관(Mask=1)'인 곳만 칠함.
        이미 칠해진 곳은 카운트하지 않음.
        return: 새롭게 칠해진 픽셀 수
        """
        z, y, x = center_pos
        r = self.brush_radius
        
        z_min, z_max = max(0, z-r), min(self.D, z+r+1)
        y_min, y_max = max(0, y-r), min(self.H, y+r+1)
        x_min, x_max = max(0, x-r), min(self.W, x+r+1)
        
        target_area = self.mask[z_min:z_max, y_min:y_max, x_min:x_max]
        visited_area = self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max]
        
        # (혈관이면서) AND (아직 안 칠한 곳)
        newly_filled_mask = (target_area == 1) & (visited_area == 0)
        new_fill_count = np.sum(newly_filled_mask)
        
        # 방문 처리 업데이트
        self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max][newly_filled_mask] = 1.0
        
        return new_fill_count

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0
        
        # 시작하자마자 현재 위치 붓칠
        self._apply_brush(self.pos)

        state = self._get_state()
        info = {"pos": self.pos, "case": self.current_case_dir.name}
        return state, info

    def step(self, action: int):
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)
        
        # 1. 볼륨 이탈 체크
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            reward = -1.0
            done = True
            return self._get_state(), reward, done, False, {"inside": False}

        # 2. 혈관 내부 체크
        inside = bool(self.mask[nz, ny, nx] > 0)

        if not inside:
            # 혈관 밖 -> 페널티 받고 종료
            reward = -1.0
            done = True
        else:
            # 3. [보상 로직] 브러쉬질 수행
            new_pixels = self._apply_brush((nz, ny, nx))
            
            if new_pixels > 0:
                # 새로운 영역을 채움 (Good!)
                # 1.0 기본 점수 + 픽셀 수 비례 추가 보너스
                reward = 1.0 + (new_pixels * 0.01)
                done = False
            else:
                # 이미 다 칠해진 곳임 (Bad - 시간 낭비)
                # 약한 페널티를 줘서 다른 곳으로 이동 유도
                reward = -0.1
                done = False

        self.pos = (nz, ny, nx)
        self.step_count += 1
        
        if self.step_count >= self.max_steps:
            done = True

        return self._get_state(), reward, done, False, {"inside": inside}


# ==========================================
# 3. 신경망 (DQN Agent)
# ==========================================

class CnnQNet(nn.Module):
    def __init__(self, in_channels=2, n_actions=6): # 입력 채널 2 (CT, Visited)
        super().__init__()
        # BatchNorm 제거 (안정성)
        self.conv1 = nn.Conv3d(in_channels, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)
        
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)

        # Flatten 후 FC
        # 32 -> 16 -> 8 -> 4 크기로 줄어듦. 채널 64.
        self.fc1 = nn.Linear(64 * 4 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, n_actions)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def sample_action(self, obs_np, epsilon):
        # Epsilon-Greedy
        if random.random() < epsilon:
            return random.randint(0, self.fc2.out_features - 1)
        
        obs_t = torch.from_numpy(obs_np).float().unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            q_out = self.forward(obs_t)
            action = q_out.argmax(dim=1).item()
        return action


# ==========================================
# 4. 학습 헬퍼 (Replay Buffer & Train)
# ==========================================

class ReplayBuffer:
    def __init__(self, buffer_limit=BUFFER_LIMIT):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for (s, a, r, s_prime, done_mask) in mini_batch:
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return (torch.tensor(np.array(s_lst), dtype=torch.float32, device=DEVICE),
                torch.tensor(a_lst, dtype=torch.int64, device=DEVICE),
                torch.tensor(r_lst, dtype=torch.float32, device=DEVICE),
                torch.tensor(np.array(s_prime_lst), dtype=torch.float32, device=DEVICE),
                torch.tensor(done_mask_lst, dtype=torch.float32, device=DEVICE))

    def size(self):
        return len(self.buffer)

def train_dqn(q_net, target_net, memory, optimizer, batch_size=BATCH_SIZE, gamma=GAMMA):
    if memory.size() < batch_size: return

    s, a, r, s_prime, done_mask = memory.sample(batch_size)

    q_out = q_net(s)
    q_a = q_out.gather(1, a)

    with torch.no_grad():
        max_q_prime = target_net(s_prime).max(1, keepdim=True)[0]
        target = r + gamma * max_q_prime * done_mask

    loss = F.smooth_l1_loss(q_a, target) # Huber Loss 사용

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# ==========================================
# 5. 메인 학습 루프 (Main Loop)
# ==========================================

def train_vessel_dqn(n_episodes=300):
    env = CTVesselEnv(DATA_ROOT, patch_size=PATCH_SIZE, max_steps=MAX_STEPS)

    q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net.load_state_dict(q_net.state_dict())

    memory = ReplayBuffer(BUFFER_LIMIT)
    optimizer = optim.Adam(q_net.parameters(), lr=LR)

    print_interval = 5
    
    print(">>> Training Started...")
    
    for ep in range(1, n_episodes + 1):
        state, info = env.reset()
        ep_reward = 0.0
        ep_steps = 0
        
        # Epsilon Decay: 1.0 -> 0.01 (전체 에피소드의 50% 구간 동안 감소)
        epsilon = max(0.01, 1.0 - (ep / (n_episodes * 0.5)))

        for t in range(MAX_STEPS):
            action = q_net.sample_action(state, epsilon)
            next_state, reward, done, _, info2 = env.step(action)

            # Done Mask: 종료 시 0.0, 진행 중 1.0
            done_mask = 0.0 if done else 1.0

            memory.put((state, action, reward, next_state, done_mask))

            state = next_state
            ep_reward += reward
            ep_steps += 1

            # 학습 (배치 단위 업데이트)
            train_dqn(q_net, target_net, memory, optimizer, BATCH_SIZE)

            if done: break
        
        # 타겟 네트워크 업데이트
        if ep % 10 == 0:
            target_net.load_state_dict(q_net.state_dict())

        # 로그 출력
        if ep % print_interval == 0:
            print(f"[Ep {ep:03d}] Score: {ep_reward:7.1f} | Steps: {ep_steps:4d} | "
                  f"Eps: {epsilon:.2f} | Case: {info2.get('case', 'NA')}")

    print(">>> Training Finished!")
    
    # 모델 저장
    save_path = "vessel_dqn_brush.pth"
    torch.save(q_net.state_dict(), save_path)
    print(f"✅ Model saved to {save_path}")

    return q_net, env


# ==========================================
# 6. 추론 및 시각화 (Inference & Visualization)
# ==========================================

def run_inference_and_save(env, q_net, save_name="result_vis.nii.gz"):
    print(f"\n--- [Inference Test] {save_name} ---")
    
    state, info = env.reset()
    case_name = info['case']
    print(f"Case: {case_name}")

    done = False
    total_reward = 0
    steps = 0
    
    while not done:
        # Epsilon=0 (Greedy)
        action = q_net.sample_action(state, epsilon=0.0)
        next_state, reward, done, _, _ = env.step(action)
        
        state = next_state
        total_reward += reward
        steps += 1
        
    print(f"Result -> Score: {total_reward:.1f}, Steps: {steps}")

    # 저장
    save_dir = Path("inference_results")
    save_dir.mkdir(exist_ok=True)

    # 1. AI가 색칠한 Volume
    visited_img = nib.Nifti1Image(env.visited_vol, affine=np.eye(4))
    nib.save(visited_img, save_dir / f"{case_name}_visited.nii.gz")

    # 2. 정답 (Target)
    target_img = nib.Nifti1Image(env.mask, affine=np.eye(4))
    nib.save(target_img, save_dir / f"{case_name}_target.nii.gz")
    
    # 3. 원본 CT
    ct_img = nib.Nifti1Image(env.ct, affine=np.eye(4))
    nib.save(ct_img, save_dir / f"{case_name}_ct.nii.gz")

    print(f"💾 Saved to {save_dir}")
    print("Use ITK-SNAP to visualize (Overlay 'visited' as Segmentation)")


# ==========================================
# 7. 실행 (Execution)
# ==========================================

if __name__ == "__main__":
    # 1. 학습 실행
    q_net, env = train_vessel_dqn(n_episodes=300)
    
    # 2. 결과 시각화 저장 (바로 확인)
    run_inference_and_save(env, q_net)

Using device: mps
Found 100 cases.
>>> Training Started...
[Ep 005] Score:    31.7 | Steps:   46 | Eps: 0.97 | Case: NA
[Ep 010] Score:    24.8 | Steps:   33 | Eps: 0.93 | Case: NA
[Ep 015] Score:    12.2 | Steps:   21 | Eps: 0.90 | Case: NA
[Ep 020] Score:    53.6 | Steps:   89 | Eps: 0.87 | Case: NA
[Ep 025] Score:   251.1 | Steps:  350 | Eps: 0.83 | Case: NA
[Ep 030] Score:    74.0 | Steps:   93 | Eps: 0.80 | Case: NA
[Ep 035] Score:    91.6 | Steps:  112 | Eps: 0.77 | Case: NA
[Ep 040] Score:     9.1 | Steps:   12 | Eps: 0.73 | Case: NA
[Ep 045] Score:   161.5 | Steps:  168 | Eps: 0.70 | Case: NA
[Ep 050] Score:     1.2 | Steps:    3 | Eps: 0.67 | Case: NA
[Ep 055] Score:   256.0 | Steps:  290 | Eps: 0.63 | Case: NA
[Ep 060] Score:   520.6 | Steps:  545 | Eps: 0.60 | Case: NA
[Ep 065] Score:   204.5 | Steps:  226 | Eps: 0.57 | Case: NA
[Ep 070] Score:   226.7 | Steps:  235 | Eps: 0.53 | Case: NA
[Ep 075] Score:   103.9 | Steps:  105 | Eps: 0.50 | Case: NA
[Ep 080] Score:   985.4 | 

KeyboardInterrupt: 

In [18]:
# === 학습 중간에 멈추고 실행하는 코드 ===

print("🛑 학습을 중단하고 현재 상태를 저장합니다...")

# 1. 현재까지 학습된 모델 저장 (이게 제일 중요!)
save_path = "vessel_dqn_early_stop.pth"
torch.save(q_net.state_dict(), save_path)
print(f"✅ 모델 저장 완료: {save_path}")

# 2. 바로 결과 뽑아보기 (Inference)
# 학습 때 안 본 데이터(Test mode)가 아니라, 일단 잘 되는지 확인하기 위해
# 현재 환경(env) 그대로 테스트해봅니다.
print("\n--- 결과 시각화 파일 생성 시작 ---")

# 시각화 함수 실행 (메모리에 있는 q_net 사용)
run_inference_and_save(env, q_net, save_name="result_early_stop.nii.gz")

print("\n🚀 확인 방법:")
print("1. 생성된 'inference_results' 폴더로 가세요.")
print("2. '..._visited.nii.gz' 와 '..._ct.nii.gz' 파일을 ITK-SNAP으로 열어보세요.")

🛑 학습을 중단하고 현재 상태를 저장합니다...
✅ 모델 저장 완료: vessel_dqn_early_stop.pth

--- 결과 시각화 파일 생성 시작 ---

--- [Inference Test] result_early_stop.nii.gz ---
Case: 01015ug_94
Result -> Score: 128.0, Steps: 128
💾 Saved to inference_results
Use ITK-SNAP to visualize (Overlay 'visited' as Segmentation)

🚀 확인 방법:
1. 생성된 'inference_results' 폴더로 가세요.
2. '..._visited.nii.gz' 와 '..._ct.nii.gz' 파일을 ITK-SNAP으로 열어보세요.


In [20]:
def run_inference_with_case_id(env, q_net, count):
    save_dir = Path("inference_results_final")
    save_dir.mkdir(exist_ok=True)
    
    print(f"--- 총 {count}회 랜덤 테스트 시작 (점수 좋은 것만 저장) ---")
    
    for i in range(count):
        state, info = env.reset()
        
        # [핵심] 현재 테스트 중인 환자 ID 가져오기
        case_id = info['case'] 
        
        done = False
        total_reward = 0
        steps = 0
        
        # 에피소드 진행 (Greedy)
        while not done:
            action = q_net.sample_action(state, epsilon=0.0)
            state, reward, done, _, _ = env.step(action)
            total_reward += reward
            steps += 1
            
        print(f"[Test {i+1}] Case: {case_id} | Score: {total_reward:.1f} | Steps: {steps}")
        
        # 점수가 100점 넘는 성공적인 케이스만 저장
        if total_reward > 100:
            # 1. AI가 그린 경로 (Visited)
            vis_name = f"{case_id}_visited_score_{int(total_reward)}.nii.gz"
            nib.save(nib.Nifti1Image(env.visited_vol, np.eye(4)), save_dir / vis_name)
            
            # 2. 원본 CT (배경용) - 같이 저장해둠
            ct_name = f"{case_id}_ct.nii.gz"
            nib.save(nib.Nifti1Image(env.ct, np.eye(4)), save_dir / ct_name)
            
            # 3. 정답 마스크 (비교용)
            target_name = f"{case_id}_target.nii.gz"
            nib.save(nib.Nifti1Image(env.mask, np.eye(4)), save_dir / target_name)
            
            print(f"   >>> 💾 저장 완료: {case_id} 세트 (visited, ct, target)")

# 10번 시도해서 잘 된 것들 뽑기
run_inference_with_case_id(env, q_net, count=10)

--- 총 10회 랜덤 테스트 시작 (점수 좋은 것만 저장) ---
[Test 1] Case: 01010ug_172 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01010ug_172 세트 (visited, ct, target)
[Test 2] Case: 01018ug_95 | Score: 115.9 | Steps: 128
   >>> 💾 저장 완료: 01018ug_95 세트 (visited, ct, target)
[Test 3] Case: 01010ug_5 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01010ug_5 세트 (visited, ct, target)
[Test 4] Case: 01010ug_5 | Score: 32.3 | Steps: 128
[Test 5] Case: 01018ug_75 | Score: 126.9 | Steps: 128
   >>> 💾 저장 완료: 01018ug_75 세트 (visited, ct, target)
[Test 6] Case: 01018ug_90 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01018ug_90 세트 (visited, ct, target)
[Test 7] Case: 01010ug_3 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01010ug_3 세트 (visited, ct, target)
[Test 8] Case: 01015ug_75 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01015ug_75 세트 (visited, ct, target)
[Test 9] Case: 01011ug_513 | Score: -1.0 | Steps: 1
[Test 10] Case: 01018ug_94 | Score: 128.0 | Steps: 128
   >>> 💾 저장 완료: 01018ug_94 세트 (visited, ct, target)


## 03. Centerline 보상 추가

- aorta 에서만 탐색하는 문제
- 모든 vessel 의 중심선을 기준으로 reward 재설계

### centerline 추출 및 저장

In [29]:
import numpy as np
import nibabel as nib
from pathlib import Path

# 🔁 skeletonize_3d 또는 skeletonize 자동 fallback
try:
    from skimage.morphology import skeletonize_3d as skeletonize_centerline
except ImportError:
    from skimage.morphology import skeletonize as skeletonize_centerline


# 🔁 새로운 데이터 경로 (largest component만 남긴 cleaned 버전)
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")

MASK_NAME = "whole_artery_cleaned.nii.gz"   # 마스크 파일 이름
SAVE_NAME = "centerline.nii.gz"     # 저장할 centerline 파일 이름


def generate_centerline(mask_volume: np.ndarray):
    """
    mask_volume: binary vessel mask (0/1)
    return: 3D skeleton (centerline) 0/1
    """
    mask_bin = (mask_volume > 0.5).astype(np.uint8)

    # skeletonize_centerline 은 skeletonize_3d 또는 fallback skeletonize
    centerline = skeletonize_centerline(mask_bin)

    return centerline.astype(np.uint8)


def process_all_cases(data_root: Path):
    case_dirs = sorted([p for p in data_root.iterdir() if p.is_dir()])

    print(f"=== Generating Centerlines for {len(case_dirs)} cases ===")

    for case in case_dirs:
        mask_path = case / MASK_NAME
        save_path = case / SAVE_NAME

        if not mask_path.exists():
            print(f"⚠️ No mask found in {case.name}, skipped.")
            continue

        # 이미 centerline이 있으면 건너뛰기
        if save_path.exists():
            print(f"✔ Already exists: {case.name}")
            continue

        try:
            mask_nii = nib.load(str(mask_path))
            mask = mask_nii.get_fdata()
        except Exception as e:
            print(f"❌ Load failed on {case.name}: {e}")
            continue

        # centerline 생성
        centerline = generate_centerline(mask)

        # 저장
        out_img = nib.Nifti1Image(centerline, affine=mask_nii.affine)
        nib.save(out_img, save_path)

        print(f"✔ Centerline saved for case: {case.name}")

    print("\n🎉 All centerlines generated!")


# 실행
process_all_cases(DATA_ROOT)

=== Generating Centerlines for 99 cases ===
✔ Centerline saved for case: 01010ug_1
✔ Centerline saved for case: 01010ug_11
✔ Centerline saved for case: 01010ug_133
✔ Centerline saved for case: 01010ug_134
✔ Centerline saved for case: 01010ug_135
✔ Centerline saved for case: 01010ug_136
✔ Centerline saved for case: 01010ug_164
✔ Centerline saved for case: 01010ug_165
✔ Centerline saved for case: 01010ug_166
✔ Centerline saved for case: 01010ug_167
✔ Centerline saved for case: 01010ug_168
✔ Centerline saved for case: 01010ug_169
✔ Centerline saved for case: 01010ug_170
✔ Centerline saved for case: 01010ug_171
✔ Centerline saved for case: 01010ug_172
✔ Centerline saved for case: 01010ug_174
✔ Centerline saved for case: 01010ug_176
✔ Centerline saved for case: 01010ug_177
✔ Centerline saved for case: 01010ug_2
✔ Centerline saved for case: 01010ug_3
✔ Centerline saved for case: 01010ug_4
✔ Centerline saved for case: 01010ug_5
✔ Centerline saved for case: 01010ug_6
✔ Centerline saved for cas

### 설계 - 데이터 로드 및 CT patch 생성

In [34]:
import os
import random
import collections
from pathlib import Path

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# =========================
# 0. 기본 설정
# =========================

DATA_ROOT   = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")
CT_NAME     = "ct.nii.gz"
MASK_NAME   = "whole_artery_cleaned.nii.gz"
CENTER_NAME = "centerline.nii.gz"

PATCH_SIZE   = (32, 32, 32)
MAX_STEPS    = 1000
BRUSH_RADIUS = 5

BATCH_SIZE   = 32
LR           = 1e-4
GAMMA        = 0.99
BUFFER_LIMIT = 50000

# 디바이스
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print("DEVICE:", DEVICE)

DEVICE: mps


In [35]:
# =========================
# 1. 로더 & 유틸
# =========================

def load_case_with_centerline(
    case_dir: Path,
    ct_name: str = CT_NAME,
    mask_name: str = MASK_NAME,
    center_name: str = CENTER_NAME,
):
    """
    case_dir 안에서
    - ct.nii.gz
    - Whole_artery.nii.gz
    - centerline.nii.gz
    를 읽어서 (ct, mask, centerline) 반환
    """
    
    ct_path   = case_dir / ct_name
    mask_path = case_dir / mask_name
    cent_path = case_dir / center_name

    if not ct_path.exists():
        raise FileNotFoundError(f"CT not found: {ct_path}")
    if not mask_path.exists():
        raise FileNotFoundError(f"Mask not found: {mask_path}")
    if not cent_path.exists():
        raise FileNotFoundError(f"Centerline not found: {cent_path}")

    ct_nii   = nib.load(str(ct_path))
    mask_nii = nib.load(str(mask_path))
    cent_nii = nib.load(str(cent_path))

    ct   = ct_nii.get_fdata().astype(np.float32)
    mask = mask_nii.get_fdata().astype(np.float32)
    cent = cent_nii.get_fdata().astype(np.float32)

    assert ct.shape == mask.shape == cent.shape, \
        f"Shape mismatch: ct{ct.shape}, mask{mask.shape}, center{cent.shape}"

    return ct, mask, cent


def normalize_ct(ct: np.ndarray, hu_min=-200.0, hu_max=300.0):
    ct = np.clip(ct, hu_min, hu_max)
    ct = (ct - hu_min) / (hu_max - hu_min + 1e-8)
    return ct.astype(np.float32)


def get_patch(volume: np.ndarray, center, patch_size=PATCH_SIZE):
    """
    volume: (D,H,W)
    center: (z,y,x)
    return: (pd,ph,pw)
    """
    D, H, W = volume.shape
    pd, ph, pw = patch_size
    zc, yc, xc = center

    z0, y0, x0 = zc - pd // 2, yc - ph // 2, xc - pw // 2
    z1, y1, x1 = z0 + pd, y0 + ph, x0 + pw

    patch = np.zeros((pd, ph, pw), dtype=volume.dtype)

    z0_src, y0_src, x0_src = max(z0, 0), max(y0, 0), max(x0, 0)
    z1_src, y1_src, x1_src = min(z1, D), min(y1, H), min(x1, W)

    z0_dst, y0_dst, x0_dst = z0_src - z0, y0_src - y0, x0_src - x0

    if z1_src > z0_src and y1_src > y0_src and x1_src > x0_src:
        patch[
            z0_dst:z0_dst + (z1_src - z0_src),
            y0_dst:y0_dst + (y1_src - y0_src),
            x0_dst:x0_dst + (x1_src - x0_src),
        ] = volume[z0_src:z1_src, y0_src:y1_src, x0_src:x1_src]

    return patch

### Env 설정

In [36]:
# =========================
# 2. Centerline RL 환경
# =========================

class CTVesselCenterEnv:
    def __init__(self, data_root: Path, patch_size=PATCH_SIZE, max_steps=MAX_STEPS):
        self.data_root = Path(data_root)
        self.patch_size = patch_size
        self.max_steps = max_steps
        self.brush_radius = BRUSH_RADIUS

        self.case_dirs = sorted([p for p in self.data_root.iterdir() if p.is_dir()])
        if not self.case_dirs:
            raise RuntimeError(f"No case folders under {self.data_root}")

        # 내부 상태
        self.ct = None
        self.mask = None
        self.centerline = None
        self.ct_norm = None

        self.visited_vol = None       # 내가 칠한 혈관 영역
        self.center_visited = None    # centerline 중 이미 밟은 곳

        self.D = self.H = self.W = None
        self.pos = None
        self.step_count = 0
        self.current_case_dir = None

    # ---------------------
    # 내부 헬퍼
    # ---------------------
    def _load_random_case(self):
        while True:
            case_dir = random.choice(self.case_dirs)
            try:
                ct, mask, cent = load_case_with_centerline(case_dir)
            except Exception as e:
                print(f"[Skip] {case_dir.name}: {e}")
                continue

            mask_bin = (mask > 0.5).astype(np.uint8)
            cent_bin = (cent > 0.5).astype(np.uint8)

            if mask_bin.sum() == 0 or cent_bin.sum() == 0:
                # 혈관이 없거나 centerline 비어있으면 스킵
                continue

            self.ct = ct
            self.mask = mask_bin
            self.centerline = cent_bin
            self.ct_norm = normalize_ct(ct)

            self.visited_vol = np.zeros_like(self.mask, dtype=np.float32)
            self.center_visited = np.zeros_like(self.centerline, dtype=np.float32)

            self.D, self.H, self.W = self.ct.shape
            self.current_case_dir = case_dir
            break

    def _sample_start_pos(self):
        """혈관 내부 아무 곳에서 시작 (나중에 aorta 주변으로 제한 가능)"""
        vessel_indices = np.argwhere(self.mask > 0)
        idx = vessel_indices[np.random.randint(len(vessel_indices))]
        return tuple(idx)  # (z,y,x)

    def _get_state(self):
        """
        state: (2, Dp, Hp, Wp)
          ch0: CT normalized
          ch1: visited_vol
        """
        ct_patch  = get_patch(self.ct_norm, self.pos, self.patch_size)
        vis_patch = get_patch(self.visited_vol, self.pos, self.patch_size)
        state = np.stack([ct_patch, vis_patch], axis=0).astype(np.float32)
        return state

    def _move(self, z, y, x, action: int):
        # 6방향 (z±1, y±1, x±1)
        deltas = [
            ( 1,  0,  0),  # +z
            (-1,  0,  0),  # -z
            ( 0,  1,  0),  # +y
            ( 0, -1,  0),  # -y
            ( 0,  0,  1),  # +x
            ( 0,  0, -1),  # -x
        ]
        dz, dy, dx = deltas[action]
        return z + dz, y + dy, x + dx

    def _brush_and_count(self, center_pos):
        """
        브러시 영역 안에서,
        - 새로 칠해진 혈관 복셀 수 반환
        - visited_vol 업데이트
        """
        z, y, x = center_pos
        r = self.brush_radius

        z_min, z_max = max(0, z-r), min(self.D, z+r+1)
        y_min, y_max = max(0, y-r), min(self.H, y+r+1)
        x_min, x_max = max(0, x-r), min(self.W, x+r+1)

        target_area  = self.mask[z_min:z_max, y_min:y_max, x_min:x_max]
        visited_area = self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max]

        newly_mask = (target_area == 1) & (visited_area == 0)
        new_voxels = np.sum(newly_mask)

        self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max][newly_mask] = 1.0

        return int(new_voxels)

    # ---------------------
    # Gym 스타일 API
    # ---------------------
    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0

        # 시작 위치도 한 번 칠해두기
        self._brush_and_count(self.pos)

        state = self._get_state()
        info = {
            "pos": self.pos,
            "case": self.current_case_dir.name,
        }
        return state, info

    def step(self, action: int):
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)

        # -------- 1) 볼륨 밖 체크 --------
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            reward = -1.0
            done = True
            info = {
                "pos": (nz, ny, nx),
                "case": self.current_case_dir.name,
                "inside_vessel": False,
                "on_centerline": False,
            }
            return self._get_state(), reward, done, False, info

        # -------- 2) 혈관 내부 체크 --------
        inside_vessel = bool(self.mask[nz, ny, nx] > 0)
        if not inside_vessel:
            reward = -1.0
            done = True
            info = {
                "pos": (nz, ny, nx),
                "case": self.current_case_dir.name,
                "inside_vessel": False,
                "on_centerline": False,
            }
            return self._get_state(), reward, done, False, info

        # -------- 3) 보상 계산 --------
        reward = 0.0
        done = False

        # 3-1. 브러시로 새로 칠한 혈관 복셀 수
        new_voxels = self._brush_and_count((nz, ny, nx))
        if new_voxels > 0:
            reward += 0.01 * new_voxels  # coverage reward

        # 3-2. centerline 보상
        on_center = bool(self.centerline[nz, ny, nx] > 0)
        if on_center:
            if self.center_visited[nz, ny, nx] == 0:
                # 처음 밟은 centerline → 큰 보상
                reward += 10.0
                self.center_visited[nz, ny, nx] = 1.0
            else:
                # 이미 밟았던 centerline → 작은 보상
                reward += 0.1
        else:
            # centerline이 아닌 곳 → 살짝 페널티
            reward -= 0.05

        self.pos = (nz, ny, nx)
        self.step_count += 1

        if self.step_count >= self.max_steps:
            done = True

        info = {
            "pos": self.pos,
            "case": self.current_case_dir.name,
            "inside_vessel": inside_vessel,
            "on_centerline": on_center,
        }

        return self._get_state(), reward, done, False, info

In [38]:
env = CTVesselCenterEnv(DATA_ROOT)
s, info = env.reset()
print("state shape:", s.shape, "case:", info)

next_s, r, done, _, info2 = env.step(0)
print("reward:", r, "done:", done, "info:", info2)

state shape: (2, 32, 32, 32) case: {'pos': (np.int64(351), np.int64(310), np.int64(164)), 'case': '01011ug_509'}
reward: -0.05 done: False info: {'pos': (np.int64(352), np.int64(310), np.int64(164)), 'case': '01011ug_509', 'inside_vessel': True, 'on_centerline': False}


### 설계 - 3D DQN

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np


# === 3D CNN Q-network ===
class CnnQNet(nn.Module):
    """
    입력:  (B, 2, 32, 32, 32)  # 2채널 (CT, visited)
    출력:  (B, 6)              # 6개 action 에 대한 Q값
    """
    def __init__(self, in_channels=2, n_actions=6):
        super().__init__()
        self.conv1 = nn.Conv3d(in_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.pool  = nn.MaxPool3d(kernel_size=2, stride=2)

        # 32 -> 16 -> 8 -> 4 로 줄어듦 (pool 3번)
        # 최종 feature map 크기: (64, 4, 4, 4)
        self.fc1 = nn.Linear(64 * 4 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, n_actions)

    def forward(self, x):
        # x: (B, 2, 32, 32, 32)
        x = self.pool(F.relu(self.conv1(x)))   # (B,16,16,16,16)
        x = self.pool(F.relu(self.conv2(x)))   # (B,32, 8, 8, 8)
        x = self.pool(F.relu(self.conv3(x)))   # (B,64, 4, 4, 4)
        x = torch.flatten(x, 1)                # (B, 64*4*4*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                        # (B, n_actions)
        return x

    def sample_action(self, obs_np: np.ndarray, epsilon: float):
        """
        obs_np : numpy state (2, 32, 32, 32)
        epsilon: epsilon-greedy 탐색 비율
        """
        # 무작위 탐색
        if random.random() < epsilon:
            return random.randint(0, self.fc2.out_features - 1)

        # greedy action (Q값 기반)
        obs_t = torch.from_numpy(obs_np).float().unsqueeze(0).to(DEVICE)  # (1,2,32,32,32)
        with torch.no_grad():
            q_values = self.forward(obs_t)  # (1,6)
            action = q_values.argmax(dim=1).item()
        return action

In [41]:
# Env 는 이미 위에서 만들어 둔 걸 그대로 사용
env = CTVesselCenterEnv(DATA_ROOT)

state, info = env.reset()
print("state shape:", state.shape, "| case:", info)

# Q-network 생성
q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)

# 더미 epsilon-greedy action 뽑기
epsilon = 0.5
action = q_net.sample_action(state, epsilon)
print("sampled action:", action)

# Q값 직접 확인
with torch.no_grad():
    s_t = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
    q_values = q_net(s_t)
    print("Q-values shape:", q_values.shape)
    print("Q-values:", q_values.cpu().numpy())

state shape: (2, 32, 32, 32) | case: {'pos': (np.int64(292), np.int64(225), np.int64(256)), 'case': '01015ug_91'}
sampled action: 5
Q-values shape: torch.Size([1, 6])
Q-values: [[-0.00940971 -0.04117823 -0.0544397   0.01041314  0.03447653  0.05736012]]


### Replay buffer

In [42]:
import collections
import random

# 리플레이 버퍼
class ReplayBuffer:
    def __init__(self, capacity=50_000):
        self.buffer = collections.deque(maxlen=capacity)

    def put(self, transition):
        """
        transition: (state, action, reward, next_state, done_mask)
        done_mask: done이면 0.0, 아니면 1.0
        """
        self.buffer.append(transition)

    def size(self):
        return len(self.buffer)

    def sample(self, batch_size):
        mini_batch = random.sample(self.buffer, batch_size)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for (s, a, r, s_prime, done_mask) in mini_batch:
            s_lst.append(s)              # (2,32,32,32) numpy
            a_lst.append([a])            # 정수 action -> (1,) 형태
            r_lst.append([r])            # 스칼라 reward
            s_prime_lst.append(s_prime)  # 다음 state
            done_mask_lst.append([done_mask])

        s      = torch.tensor(np.array(s_lst),      dtype=torch.float32, device=DEVICE)
        a      = torch.tensor(a_lst,                dtype=torch.int64,   device=DEVICE)
        r      = torch.tensor(r_lst,                dtype=torch.float32, device=DEVICE)
        sPrime = torch.tensor(np.array(s_prime_lst),dtype=torch.float32, device=DEVICE)
        d_mask = torch.tensor(done_mask_lst,        dtype=torch.float32, device=DEVICE)

        return s, a, r, sPrime, d_mask

### DQN 학습

In [45]:
#### test 코드

import torch.optim as optim
import torch.nn.functional as F

GAMMA      = 0.99
BATCH_SIZE = 32

def train_dqn(q_net, target_net, memory, optimizer, batch_size=BATCH_SIZE, gamma=GAMMA):
    """
    리플레이 버퍼에서 미니배치를 꺼내서
    DQN loss를 계산하고 한 번 gradient step 하는 함수.
    """
    if memory.size() < batch_size:
        return None  # 샘플이 아직 부족하면 학습하지 않음

    s, a, r, s_prime, done_mask = memory.sample(batch_size)

    # 현재 Q(s,a)
    q_out = q_net(s)              # (B, n_actions)
    q_a   = q_out.gather(1, a)    # (B, 1)

    # 타깃 Q = r + gamma * max_a' Q_target(s', a') * done_mask
    with torch.no_grad():
        max_q_prime = target_net(s_prime).max(1, keepdim=True)[0]  # (B,1)
        target = r + gamma * max_q_prime * done_mask               # (B,1)

    loss = F.smooth_l1_loss(q_a, target)  # Huber loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

In [44]:
# 1) 환경 & 네트워크 & 버퍼 & 옵티마이저 준비
env = CTVesselCenterEnv(DATA_ROOT)
q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
target_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
target_net.load_state_dict(q_net.state_dict())

memory = ReplayBuffer(capacity=50_000)
optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

# 2) 환경에서 몇 스텝 굴리면서 버퍼 채우기
state, info = env.reset()
epsilon = 1.0  # 초반엔 랜덤하게 많이 탐색

for t in range(50):   # 50 step 정도만
    action = q_net.sample_action(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)

    done_mask = 0.0 if done else 1.0
    memory.put((state, action, reward, next_state, done_mask))

    state = next_state
    if done:
        state, info = env.reset()

print("Buffer size after rollout:", memory.size())

# 3) DQN 한 번 업데이트
loss_val = train_dqn(q_net, target_net, memory, optimizer)

print("One training step loss:", loss_val)

Buffer size after rollout: 50
One training step loss: 0.371529757976532


In [47]:
#### 하이퍼 파라미터 및 메인 학습 루프

import torch.optim as optim

# ==== 하이퍼파라미터 ====
NUM_EPISODES      = 200          # 전체 에피소드 수
MAX_STEPS_PER_EP  = 1000         # 에피소드당 최대 스텝
EPS_START         = 1.0          # epsilon 시작값
EPS_END           = 0.05         # epsilon 최소값
EPS_DECAY         = 0.99         # 매 에피소드 후 epsilon *= EPS_DECAY
TARGET_UPDATE_INT = 5           # 타깃 네트워크 업데이트 주기 (에피소드 단위)
PRINT_INTERVAL    = 5           # 로그 출력 주기


def train_vessel_dqn_centerline(
    num_episodes=NUM_EPISODES,
    max_steps_per_ep=MAX_STEPS_PER_EP,
):
    # 1) 환경, 네트워크, 버퍼, 옵티마이저 준비
    env = CTVesselCenterEnv(DATA_ROOT)

    q_net      = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net.load_state_dict(q_net.state_dict())

    memory    = ReplayBuffer(capacity=50_000)
    optimizer = optim.Adam(q_net.parameters(), lr=1e-4)

    epsilon = EPS_START

    print(">>> Training started...")
    reward_history = []

    for ep in range(1, num_episodes + 1):
        state, info = env.reset()
        ep_reward = 0.0
        ep_steps  = 0
        last_info = info

        while True:
            # 1) epsilon-greedy 행동 선택
            action = q_net.sample_action(state, epsilon)

            # 2) 환경 한 스텝 진행
            next_state, reward, done, _, step_info = env.step(action)
            last_info = step_info  # 마지막 step의 case 정보용

            # 3) transition 버퍼에 저장
            done_mask = 0.0 if done else 1.0
            memory.put((state, action, reward, next_state, done_mask))

            state = next_state
            ep_reward += reward
            ep_steps  += 1

            # 4) DQN 업데이트 (경험이 어느 정도 쌓이면)
            train_dqn(q_net, target_net, memory, optimizer)

            # 5) 에피소드 종료 조건
            if done or ep_steps >= max_steps_per_ep:
                break

        # 6) epsilon decay
        epsilon = max(EPS_END, epsilon * EPS_DECAY)

        # 7) 타깃 네트워크 동기화
        if ep % TARGET_UPDATE_INT == 0:
            target_net.load_state_dict(q_net.state_dict())

        reward_history.append(ep_reward)

        # 8) 로그 출력
        if ep % PRINT_INTERVAL == 0:
            case_name = last_info.get("case", "NA") if isinstance(last_info, dict) else "NA"
            avg_recent = sum(reward_history[-PRINT_INTERVAL:]) / len(reward_history[-PRINT_INTERVAL:])
            print(
                f"[Ep {ep:03d}] "
                f"reward={ep_reward:7.1f} (avg {avg_recent:6.1f}), "
                f"steps={ep_steps:4d}, "
                f"eps={epsilon:4.2f}, "
                f"buffer={memory.size():5d}, "
                f"case={case_name}"
            )

    print(">>> Training finished!")
    return q_net, target_net, env, reward_history

In [49]:
q_net, target_net, env, reward_hist = train_vessel_dqn_centerline(
    num_episodes = NUM_EPISODES,
    max_steps_per_ep = MAX_STEPS_PER_EP,
)

>>> Training started...
[Ep 005] reward=   -1.1 (avg   29.7), steps=   2, eps=0.95, buffer=  839, case=01015ug_91
[Ep 010] reward=   10.6 (avg   74.7), steps=  60, eps=0.90, buffer= 1305, case=01015ug_91
[Ep 015] reward=   62.8 (avg   34.2), steps= 278, eps=0.86, buffer= 1928, case=01018ug_97
[Ep 020] reward=   17.3 (avg   44.5), steps=  50, eps=0.82, buffer= 2517, case=01015ug_72
[Ep 025] reward=    0.8 (avg   33.9), steps=   9, eps=0.78, buffer= 2919, case=01011ug_530
[Ep 030] reward=  122.7 (avg   60.4), steps= 232, eps=0.74, buffer= 3502, case=01011ug_515
[Ep 035] reward=   48.0 (avg   32.5), steps= 107, eps=0.70, buffer= 3865, case=01010ug_133
[Ep 040] reward=  281.4 (avg  129.9), steps= 547, eps=0.67, buffer= 4990, case=01011ug_501
[Ep 045] reward=  262.6 (avg  135.4), steps= 566, eps=0.64, buffer= 6265, case=01011ug_508
[Ep 050] reward=  483.2 (avg  395.6), steps= 729, eps=0.61, buffer= 9305, case=01010ug_174
[Ep 055] reward=  776.1 (avg  497.2), steps=1000, eps=0.58, buffer=134

KeyboardInterrupt: 

In [51]:
torch.save(q_net.state_dict(), "checkpoint_centerline.pth")
print("saved!")

saved!


In [62]:
#### 학습된 가중치 불러오기 테스트
# 1) 모델 생성
q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)

# 2) 저장된 가중치 로드
state_dict = torch.load("checkpoint_centerline.pth", map_location=DEVICE)
q_net.load_state_dict(state_dict)

# 3) inference 모드 설정
q_net.eval()

print("✅ trained weights loaded successfully")

✅ trained weights loaded successfully


In [70]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# --------------------------------------------------
# 1) greedy rollout (ε=0) 로 한 케이스에서 trajectory 뽑기
# --------------------------------------------------
def rollout_greedy(env, q_net, max_steps=500):
    """
    env  : CTVesselCenterEnv (이미 DATA_ROOT 로 만들어져 있다고 가정)
    q_net: 학습된 CnnQNet (가중치 load 완료 상태)
    """
    state, info = env.reset()          # 랜덤 케이스 하나 선택 + 시작점 샘플
    traj = [tuple(info["pos"])]        # (z, y, x)
    done = False
    steps = 0

    while not done and steps < max_steps:
        action = q_net.sample_action(state, epsilon=0.0)  # 순수 greedy
        state, reward, done, _, info2 = env.step(action)
        traj.append(tuple(info2["pos"]))
        steps += 1

    traj = np.array(traj)
    return traj, info


# --------------------------------------------------
# 2) centerline + trajectory 3D plot (한 케이스)
# --------------------------------------------------
def plot_centerline_trajectory(env, q_net,
                               max_steps=500,
                               local_only=True,
                               pad=40):
    """
    local_only=True  : trajectory 주변만 잘라서 centerline 표시
    local_only=False : 전체 centerline 표시
    """
    traj, info0 = rollout_greedy(env, q_net, max_steps=max_steps)
    case_name = info0["case"]

    # env 안에 있는 centerline 볼륨 가져오기
    centerline_vol = getattr(env, "centerline_vol", env.centerline)
    cl_idx = np.argwhere(centerline_vol > 0)   # (Nc, 3) : (z, y, x)

    # ---- trajectory 주변만 보기 옵션 ----
    if local_only:
        z_min, y_min, x_min = traj.min(axis=0) - pad
        z_max, y_max, x_max = traj.max(axis=0) + pad

        mask = (
            (cl_idx[:, 0] >= z_min) & (cl_idx[:, 0] <= z_max) &
            (cl_idx[:, 1] >= y_min) & (cl_idx[:, 1] <= y_max) &
            (cl_idx[:, 2] >= x_min) & (cl_idx[:, 2] <= x_max)
        )
        cl_plot = cl_idx[mask]
    else:
        cl_plot = cl_idx

    # ---- 3D figure 생성 ----
    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection="3d")

    # centerline (옅은 점)
    if len(cl_plot) > 0:
        ax.scatter(
            cl_plot[:, 2], cl_plot[:, 1], cl_plot[:, 0],
            s=1, alpha=0.05, label="centerline"
        )

    # trajectory (굵은 선)
    ax.plot(
        traj[:, 2], traj[:, 1], traj[:, 0],
        linewidth=3, color="blue", label="trajectory"
    )

    # 시작 / 끝점
    ax.scatter(traj[0, 2],  traj[0, 1],  traj[0, 0],
               s=40, c="green", label="start")
    ax.scatter(traj[-1, 2], traj[-1, 1], traj[-1, 0],
               s=40, c="red",   label="end")

    ax.set_xlabel("x (col)")
    ax.set_ylabel("y (row)")
    ax.set_zlabel("z (slice)")
    ax.set_title(f"3D trajectory on centerline | case {case_name}, len={len(traj)}")
    ax.legend(loc="upper left")
    plt.tight_layout()

    print(f"[plot] case={case_name}, traj_len={len(traj)}")
    return fig, case_name


# --------------------------------------------------
# 3) 랜덤 10케이스 시각화 + PNG 저장
# --------------------------------------------------
def visualize_random_cases(env, q_net,
                           num_cases=10,
                           max_steps=500,
                           save_dir="results/traj_10cases",
                           local_only=True):
    os.makedirs(save_dir, exist_ok=True)

    for i in range(num_cases):
        fig, case_name = plot_centerline_trajectory(
            env, q_net,
            max_steps=max_steps,
            local_only=local_only
        )

        # 파일 이름: traj_00_케이스이름.png 이런 식으로 저장
        save_path = os.path.join(save_dir, f"traj_{i:02d}_{case_name}.png")
        fig.savefig(save_path, dpi=150)
        plt.close(fig)
        print(f"  → saved: {save_path}")


# --------------------------------------------------
# 4) 실제 실행
#    (q_net, env 는 이미 위에서 생성 + 가중치 load 가 끝난 상태라고 가정)
# --------------------------------------------------
visualize_random_cases(
    env,
    q_net,
    num_cases=10,          # 보고 싶은 케이스 수
    max_steps=500,         # 한 trajectory 최대 길이
    save_dir="results/traj_10cases",
    local_only=True        # 주변만 볼지 / 전체 centerline 볼지
)

[plot] case=01015ug_100, traj_len=8
  → saved: results/traj_10cases/traj_00_01015ug_100.png
[plot] case=01011ug_527, traj_len=16
  → saved: results/traj_10cases/traj_01_01011ug_527.png
[plot] case=01011ug_508, traj_len=9
  → saved: results/traj_10cases/traj_02_01011ug_508.png
[plot] case=01010ug_1, traj_len=2
  → saved: results/traj_10cases/traj_03_01010ug_1.png
[plot] case=01018ug_86, traj_len=2
  → saved: results/traj_10cases/traj_04_01018ug_86.png
[plot] case=01018ug_90, traj_len=27
  → saved: results/traj_10cases/traj_05_01018ug_90.png
[plot] case=01011ug_516, traj_len=24
  → saved: results/traj_10cases/traj_06_01011ug_516.png
[plot] case=01010ug_172, traj_len=11
  → saved: results/traj_10cases/traj_07_01010ug_172.png
[plot] case=01015ug_89, traj_len=5
  → saved: results/traj_10cases/traj_08_01015ug_89.png
[plot] case=01011ug_526, traj_len=4
  → saved: results/traj_10cases/traj_09_01011ug_526.png


In [71]:
state, _ = env.reset()
q_values = q_net(torch.from_numpy(state).float().unsqueeze(0).to(DEVICE))
print(q_values.cpu().detach().numpy())

[[ 0.06144293 -0.05346137  0.00885756  0.04749939 -0.02548346  0.02451901]]


In [76]:
# 여러 state에서 Q값 패턴이 똑같은지 보기

import numpy as np
from collections import Counter

best_actions = []
for _ in range(100):
    s, _ = env.reset()
    q = q_net(torch.from_numpy(s).float().unsqueeze(0).to(DEVICE))
    a = q.argmax(dim=1).item()
    best_actions.append(a)

print("greedy best action counts:", Counter(best_actions))

greedy best action counts: Counter({0: 98, 3: 2})


In [75]:
# 고정된 상태에서 greedy action 분포 확인
counts = np.zeros(6, dtype=int)
s, _ = env.reset()
for _ in range(1000):
    a = q_net.sample_action(s, epsilon=0.0)
    counts[a] += 1
print("greedy counts (고정 state):", counts)

greedy counts (고정 state): [   0    0    0 1000    0    0]


In [74]:
# 같은 구조의 랜덤 초기화 네트워크
rand_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
s, _ = env.reset()

q_trained = q_net(torch.from_numpy(s).float().unsqueeze(0).to(DEVICE))
q_random  = rand_net(torch.from_numpy(s).float().unsqueeze(0).to(DEVICE))

print("trained:", q_trained.cpu().detach().numpy())
print("random :", q_random.cpu().detach().numpy())

trained: [[ 0.06249352 -0.05253318  0.00406158  0.04704229 -0.02798536  0.02615421]]
random : [[-0.03136034  0.02224334  0.02839084  0.06291301  0.05760475  0.01807821]]


## 04. CNN 구조 변경 및 sparse 한 reward 수정
- centerline 방향과 action 방향에 따른 cos similarity 기반 reward 추가 

### 데이터 분할

In [81]:
from pathlib import Path
import random

DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")
SPLIT_DIR = DATA_ROOT / "splits"
SPLIT_DIR.mkdir(exist_ok=True)

CT_NAME   = "ct.nii.gz"
MASK_NAME = "whole_artery_cleaned.nii.gz"

def create_splits(
    data_root: Path,
    train_ratio: float = 0.7,
    val_ratio: float = 0.15,
    seed: int = 42,
):
    """
    CT/mask 파일이 있는 폴더만 케이스로 보고 train/val/test split 생성
    """
    # 🔍 1) 케이스 폴더 수집 (ct + mask 있는 폴더만)
    case_dirs = []
    for d in data_root.iterdir():
        if not d.is_dir():
            continue
        ct_path   = d / CT_NAME
        mask_path = d / MASK_NAME
        if ct_path.exists() and mask_path.exists():
            case_dirs.append(d)

    case_dirs = sorted(case_dirs)
    case_names = [d.name for d in case_dirs]
    print(f"총 케이스 수: {len(case_names)}")

    # 2) 셔플
    random.seed(seed)
    random.shuffle(case_names)

    # 3) 비율대로 나누기
    n_total = len(case_names)
    n_train = int(n_total * train_ratio)
    n_val   = int(n_total * val_ratio)

    train_ids = case_names[:n_train]
    val_ids   = case_names[n_train:n_train+n_val]
    test_ids  = case_names[n_train+n_val:]

    # 4) 저장
    def _save(split_name, ids):
        path = SPLIT_DIR / f"{split_name}.txt"
        with open(path, "w") as f:
            for cid in ids:
                f.write(cid + "\n")
        print(f"{split_name:>5}: {len(ids):3d}개 → {path}")

    _save("train", train_ids)
    _save("val",   val_ids)
    _save("test",  test_ids)

    return train_ids, val_ids, test_ids

# 다시 생성
train_ids, val_ids, test_ids = create_splits(DATA_ROOT, train_ratio=0.7, val_ratio=0.15, seed=2025)

print("\n예시 train 5개:", train_ids[:5])
print("예시  val 5개:", val_ids[:5])
print("예시 test 5개:", test_ids[:5])

총 케이스 수: 99
train:  69개 → /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/splits/train.txt
  val:  14개 → /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/splits/val.txt
 test:  16개 → /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/splits/test.txt

예시 train 5개: ['01018ug_73', '01011ug_506', '01015ug_89', '01015ug_100', '01015ug_72']
예시  val 5개: ['01018ug_86', '01018ug_72', '01018ug_87', '01010ug_8', '01018ug_85']
예시 test 5개: ['01010ug_136', '01010ug_170', '01011ug_528', '01018ug_88', '01010ug_166']


### 기본설정 / 데이터 import / CT 전처리

In [96]:
# ============================================================
# 0. Import & 기본 설정
# ============================================================
import os
import random
from pathlib import Path
from typing import List, Tuple, Dict

import numpy as np
import nibabel as nib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import deque

print("torch:", torch.__version__)

# ----- 데이터 / split 경로 -----
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")  # ✅ 필요하면 여기만 수정
SPLIT_DIR = DATA_ROOT / "splits"  # train.txt, val.txt, test.txt 가 있는 폴더

CT_NAME        = "ct.nii.gz"
MASK_NAME      = "whole_artery_cleaned.nii.gz"
CENTERLINE_NAME = "centerline.nii.gz"

# ----- Device 선택 -----
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(f"Using device: {DEVICE}")


torch: 2.9.1
Using device: mps


In [97]:
# ============================================================
# 1. split 로 case 목록 읽기
# ============================================================

def read_split_ids(split_dir: Path, split_name: str) -> List[str]:
    """
    split_name: 'train' / 'val' / 'test'
    txt 파일에서 case_id 리스트를 읽어옴.
    """
    path = split_dir / f"{split_name}.txt"
    if not path.exists():
        raise FileNotFoundError(f"{path} not found")

    ids = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                ids.append(line)
    return ids


def get_case_dirs(data_root: Path, case_ids: List[str]) -> List[Path]:
    case_dirs = []
    for cid in case_ids:
        cdir = data_root / cid
        if not cdir.is_dir():
            print(f"⚠️ case dir missing: {cdir}")
            continue
        case_dirs.append(cdir)
    return case_dirs


# ============================================================
# 2. NIfTI 로딩 & 전처리 유틸
# ============================================================

def load_case(case_dir: Path,
              ct_name: str = CT_NAME,
              mask_name: str = MASK_NAME,
              center_name: str = CENTERLINE_NAME):
    """ct / mask / centerline NIfTI를 모두 읽어옴."""
    ct_path   = case_dir / ct_name
    mask_path = case_dir / mask_name
    ctr_path  = case_dir / center_name

    if not ct_path.exists():   raise FileNotFoundError(ct_path)
    if not mask_path.exists(): raise FileNotFoundError(mask_path)
    if not ctr_path.exists():  raise FileNotFoundError(ctr_path)

    ct_nii   = nib.load(str(ct_path))
    mask_nii = nib.load(str(mask_path))
    ctr_nii  = nib.load(str(ctr_path))

    ct   = ct_nii.get_fdata().astype(np.float32)
    mask = mask_nii.get_fdata().astype(np.float32)
    ctr  = ctr_nii.get_fdata().astype(np.float32)

    assert ct.shape == mask.shape == ctr.shape, f"shape mismatch in {case_dir.name}"

    return ct, mask, ctr


def normalize_ct(ct: np.ndarray, hu_min=-200.0, hu_max=300.0) -> np.ndarray:
    """HU windowing → 0~1"""
    ct = np.clip(ct, hu_min, hu_max)
    ct = (ct - hu_min) / (hu_max - hu_min + 1e-8)
    return ct.astype(np.float32)


def get_patch(volume: np.ndarray, center: Tuple[int, int, int],
              patch_size: Tuple[int, int, int] = PATCH_SIZE) -> np.ndarray:
    """
    volume: (D,H,W)
    center: (z,y,x)
    return: (pd,ph,pw)
    """
    D, H, W = volume.shape
    pd, ph, pw = patch_size
    zc, yc, xc = center

    z0, y0, x0 = zc - pd // 2, yc - ph // 2, xc - pw // 2
    z1, y1, x1 = z0 + pd, y0 + ph, x0 + pw

    patch = np.zeros((pd, ph, pw), dtype=volume.dtype)

    z0s, y0s, x0s = max(z0, 0), max(y0, 0), max(x0, 0)
    z1s, y1s, x1s = min(z1, D), min(y1, H), min(x1, W)

    z0d, y0d, x0d = z0s - z0, y0s - y0, x0s - x0

    if z1s > z0s and y1s > y0s and x1s > x0s:
        patch[z0d:z0d+(z1s-z0s),
              y0d:y0d+(y1s-y0s),
              x0d:x0d+(x1s-x0s)] = volume[z0s:z1s, y0s:y1s, x0s:x1s]

    return patch

### Centerline 방향 추정 + cosine 계산

In [87]:
## Cos similarity test

import numpy as np
from scipy.spatial import cKDTree  # 이미 설치되어 있을 거고, 없으면 pip install scipy

# ---------------------------------------------------
# 1) centerline 좌표 뽑아서 KDTree 만들기
# ---------------------------------------------------

# env 안에 centerline 볼륨이 있다고 가정
centerline_vol = env.centerline if hasattr(env, "centerline") else env.centerline_vol

# centerline_vol > 0 인 복셀들의 (z,y,x) 좌표들
cl_coords = np.argwhere(centerline_vol > 0)   # shape: (N, 3)

print(f"Centerline points: {cl_coords.shape[0]} voxels")

# 최근접 탐색을 위한 KDTree
cl_tree = cKDTree(cl_coords)


# ---------------------------------------------------
# 2) 해당 위치 근처의 centerline 방향(tangent) 추정 함수
#    - PCA 방식: 주변 몇 개의 centerline 점으로 주성분 방향(첫 PC)을 tangent 로 사용
# ---------------------------------------------------
def estimate_tangent(centerline_coords: np.ndarray,
                     centerline_tree: cKDTree,
                     pos: tuple,
                     k: int = 5) -> np.ndarray:
    """
    pos: (z, y, x) voxel 좌표
    k  : 주변 몇 개의 centerline 점을 사용할지 (local window 크기)
    return: tangent vector (길이 3의 numpy array)
    """
    pos = np.array(pos, dtype=float)

    # centerline 상에서 pos 에 가장 가까운 k개의 점 찾기
    k = min(k, len(centerline_coords))
    dists, idxs = centerline_tree.query(pos, k=k)

    # idxs 가 scalar 가 될 수 있어서 보정
    if np.isscalar(idxs):
        local_pts = centerline_coords[[idxs]]  # (1,3)
    else:
        local_pts = centerline_coords[idxs]    # (k,3)

    # 주변 점들이 최소 2개는 있어야 방향을 추정할 수 있음
    if local_pts.shape[0] < 2:
        # 방향이 정의 안 되면 0벡터 반환
        return np.zeros(3, dtype=float)

    # PCA(SVD) 로 1차 주성분 = 혈관 진행 방향 근사
    mean = local_pts.mean(axis=0)
    X = local_pts - mean
    # (k,3) -> SVD
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    tangent = Vt[0]   # shape: (3,)  첫 번째 주성분

    return tangent


# ---------------------------------------------------
# 3) 이동 벡터와 tangent 간 cosine similarity 계산 함수
# ---------------------------------------------------
def cosine_to_centerline(prev_pos, next_pos, tangent_vec) -> float:
    """
    prev_pos: (z,y,x) 이전 위치
    next_pos: (z,y,x) 다음 위치
    tangent_vec: centerline 방향 벡터(3,)
    return: cosine similarity [-1, 1]
    """
    prev_pos = np.array(prev_pos, dtype=float)
    next_pos = np.array(next_pos, dtype=float)
    move_vec = next_pos - prev_pos  # agent 이동 방향

    # 길이가 0인 벡터면 정의 불가 → 0으로 처리
    if np.linalg.norm(move_vec) == 0 or np.linalg.norm(tangent_vec) == 0:
        return 0.0

    v1 = move_vec / np.linalg.norm(move_vec)
    v2 = tangent_vec / np.linalg.norm(tangent_vec)
    cos = float(np.dot(v1, v2))

    # 수치오차 방지용 clip
    cos = float(np.clip(cos, -1.0, 1.0))
    return cos


# ---------------------------------------------------
# 4) 간단 테스트: env에서 한 스텝 굴려보고 cosine 값 찍어보기
# ---------------------------------------------------

# 1) 환경 리셋 → 시작 위치
state, info = env.reset()
p0 = tuple(info["pos"])   # (z,y,x)
print("Start pos:", p0, "| case:", info.get("case", "NA"))

# 2) 예시로 action 하나 선택 (임시로 0번 액션)
#   - 여기서는 단순 테스트용이므로 q_net 없이 random 으로도 가능
test_action = 0  # 또는 random.randint(0,5)

# env 내부 step을 써서 next_pos 얻기
state2, reward, done, _, info2 = env.step(test_action)
p1 = tuple(info2["pos"])  # (nz,ny,nx)

print("Next pos:", p1)

# 3) tangent vector 추정 (p1 주변 centerline 방향)
tangent = estimate_tangent(cl_coords, cl_tree, p1, k=5)
print("Tangent vec near p1:", tangent)

# 4) 코사인 값 계산
cos_val = cosine_to_centerline(p0, p1, tangent)
print("Cosine(centerline_dir, move_dir) =", cos_val)

Centerline points: 1642 voxels
Start pos: (np.int64(271), np.int64(232), np.int64(42)) | case: 01018ug_105
Next pos: (np.int64(272), np.int64(232), np.int64(42))
Tangent vec near p1: [ 0.16741001  0.33026265 -0.92892436]
Cosine(centerline_dir, move_dir) = 0.16741000551157248


In [98]:
# =========================
# 2. Centerline 관련 유틸
# =========================

def build_centerline_kdtree(centerline_vol: np.ndarray):
    """
    centerline_vol > 0 인 좌표들로 KDTree 생성
    return: (coords, tree)
      - coords: (N,3) array (z, y, x)
    """
    coords = np.argwhere(centerline_vol > 0)
    if coords.shape[0] == 0:
        raise RuntimeError("Centerline volume is empty.")
    tree = cKDTree(coords)
    return coords, tree


def estimate_tangent(coords: np.ndarray,
                     tree: cKDTree,
                     pos: tuple,
                     k: int = 5):
    """
    현재 위치 pos 근처의 centerline 점들을 이용해 tangent vector 추정
    coords: (N,3)
    pos: (z, y, x)
    """
    pos_arr = np.array(pos, dtype=np.float32)
    dist, idx = tree.query(pos_arr, k=k)
    neighbors = coords[idx]    # (k, 3)

    # 중심에서 이웃들의 방향 평균
    vecs = neighbors - pos_arr[None, :]
    v = vecs.mean(axis=0)

    norm = np.linalg.norm(v) + 1e-8
    return v / norm


def cosine_to_centerline(prev_pos: tuple,
                         new_pos: tuple,
                         tangent_vec: np.ndarray):
    """
    prev_pos -> new_pos 로의 이동 방향과 centerline tangent간 cosine 계산
    """
    p0 = np.array(prev_pos, dtype=np.float32)
    p1 = np.array(new_pos,  dtype=np.float32)
    move_vec = p1 - p0

    mv_norm = np.linalg.norm(move_vec)
    if mv_norm < 1e-8:
        return 0.0

    move_vec = move_vec / (mv_norm + 1e-8)
    t = tangent_vec / (np.linalg.norm(tangent_vec) + 1e-8)

    cos_val = float(np.dot(move_vec, t))
    return cos_val  # -1 ~ +1

### Env 설정

In [99]:
# ----- RL / DQN 설정 -----
PATCH_SIZE = (32, 32, 32)
BRUSH_RADIUS = 1         # centerline 주변 살짝만 칠하기
MAX_STEPS_PER_EP = 800

GAMMA = 0.99
LR = 1e-4
BATCH_SIZE = 32
BUFFER_LIMIT = 50000

NUM_EPISODES = 300
TARGET_UPDATE_EP = 10        # 몇 episode마다 target net 동기화
VAL_INTERVAL_EP = 20         # 몇 episode마다 validation 수행

EPS_START = 1.0
EPS_END   = 0.05
EPS_DECAY_EP = int(NUM_EPISODES * 0.7)   # 70% 동안 선형 감소

# ----- 보상 함수 관련 -----
R_DIR_SCALE = 1.0   # 일단 1.0부터 시작, 필요하면 2.0, 3.0으로 키워보기
R_OUT       = -1.0  # 밖으로 나갔을 때 패널티


In [100]:
# =========================
# 3. Env 정의
# =========================

class CTVesselCenterEnv:
    """
    state: (2, 32, 32, 32)
      - ch0: normalized CT patch
      - ch1: visited volume patch (내가 segmentation 한 결과)

    reward:
      - volume 밖 / vessel 밖 : -1, done
      - vessel 내부:
          reward = |cos(centerline_dir, move_dir)|   (0 ~ 1.0)
      - brush(visited_vol)은 state 업데이트 용도만, reward에는 영향 없음
    """
    def __init__(self,
                 data_root: Path,
                 patch_size=PATCH_SIZE,
                 max_steps=MAX_STEPS,
                 brush_radius=BRUSH_RADIUS):

        self.data_root    = Path(data_root)
        self.patch_size   = patch_size
        self.max_steps    = max_steps
        self.brush_radius = brush_radius

        self.case_dirs = sorted([p for p in self.data_root.iterdir() if p.is_dir()])
        if not self.case_dirs:
            raise RuntimeError(f"No case folders found under {self.data_root}")

        print(f"Found {len(self.case_dirs)} cases under {self.data_root}")

        # episode-specific 상태
        self.ct            = None
        self.mask          = None
        self.centerline    = None
        self.ct_norm       = None
        self.visited_vol   = None

        self.D = self.H = self.W = None
        self.pos          = None
        self.step_count   = 0
        self.current_case_dir = None

        # centerline KDTree용
        self.centerline_coords = None
        self.centerline_tree   = None

    # ---------- 내부 유틸 ----------

    def _load_random_case(self):
        """데이터셋에서 랜덤 케이스 로드"""
        while True:
            case_dir = random.choice(self.case_dirs)
            try:
                ct, mask, cl = load_case(case_dir)
            except Exception as e:
                print(f"[WARN] skip {case_dir.name} - load error: {e}")
                continue

            if mask.sum() == 0 or cl.sum() == 0:
                print(f"[WARN] skip {case_dir.name} - empty mask/centerline")
                continue

            self.ct         = ct
            self.mask       = mask.astype(np.uint8)
            self.centerline = cl.astype(np.uint8)
            self.ct_norm    = normalize_ct(ct)

            self.visited_vol = np.zeros_like(self.mask, dtype=np.float32)

            self.D, self.H, self.W = self.ct.shape
            self.current_case_dir = case_dir

            # centerline KDTree
            coords, tree = build_centerline_kdtree(self.centerline)
            self.centerline_coords = coords
            self.centerline_tree   = tree

            break  # 성공적으로 로드했으면 종료

    def _sample_start_pos(self):
        """시작 위치: centerline 위에서 랜덤 선택"""
        cl_indices = self.centerline_coords
        idx = np.random.randint(len(cl_indices))
        return tuple(cl_indices[idx])

    def _get_state(self):
        """2채널 state 반환: [CT patch, visited patch]"""
        ct_patch  = get_patch(self.ct_norm,    self.pos, self.patch_size)
        vis_patch = get_patch(self.visited_vol, self.pos, self.patch_size)
        state = np.stack([ct_patch, vis_patch], axis=0).astype(np.float32)
        return state

    def _move(self, z, y, x, action):
        """
        action: 0~5 (6방향)
          0: +z, 1: -z
          2: +y, 3: -y
          4: +x, 5: -x
        """
        deltas = [
            (1, 0, 0),   # +z
            (-1, 0, 0),  # -z
            (0, 1, 0),   # +y
            (0, -1, 0),  # -y
            (0, 0, 1),   # +x
            (0, 0, -1),  # -x
        ]
        dz, dy, dx = deltas[action]
        return z + dz, y + dy, x + dx

    def _apply_brush_vessel(self, center_pos):
        """
        vessel mask 기준으로 붓칠:
          - Whole_artery mask == 1 이면서
          - 아직 visited == 0 인 voxel만 1로 채운다
        return: 새로 채운 voxel 개수 (debug용)
        """
        z, y, x = center_pos
        r = self.brush_radius

        z_min, z_max = max(0, z-r), min(self.D, z+r+1)
        y_min, y_max = max(0, y-r), min(self.H, y+r+1)
        x_min, x_max = max(0, x-r), min(self.W, x+r+1)

        target  = self.mask[z_min:z_max, y_min:y_max, x_min:x_max]
        visited = self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max]

        newly = (target == 1) & (visited == 0)
        new_count = int(newly.sum())

        visited[newly] = 1.0
        self.visited_vol[z_min:z_max, y_min:y_max, x_min:x_max] = visited
        return new_count

    # ---------- Gym 스타일 인터페이스 ----------

    def reset(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        self._load_random_case()
        self.pos = self._sample_start_pos()
        self.step_count = 0

        # 시작 위치도 한번 붓칠
        self._apply_brush_vessel(self.pos)

        state = self._get_state()
        info = {
            "pos":  self.pos,
            "case": self.current_case_dir.name,
        }
        return state, info

    def step(self, action: int):
        z, y, x = self.pos
        nz, ny, nx = self._move(z, y, x, action)

        # 1) 볼륨 밖 → 즉시 종료
        if not (0 <= nz < self.D and 0 <= ny < self.H and 0 <= nx < self.W):
            reward = -1.0
            done = True
            return self._get_state(), reward, done, False, {
                "pos": self.pos,
                "case": self.current_case_dir.name,
                "reason": "out_of_volume"
            }

        # 2) vessel mask 밖 → 즉시 종료
        if self.mask[nz, ny, nx] == 0:
            self.pos = (nz, ny, nx)
            reward = -1.0
            done = True
            return self._get_state(), reward, done, False, {
                "pos": self.pos,
                "case": self.current_case_dir.name,
                "reason": "out_of_vessel"
            }

        # 3) vessel 내부 (정상 이동)
        # (A) centerline tangent 추정
        tangent = estimate_tangent(
            self.centerline_coords,
            self.centerline_tree,
            (nz, ny, nx),
            k=5
        )

        # (B) 이동 방향과의 cosine
        cos_val = cosine_to_centerline(
            (z, y, x),
            (nz, ny, nx),
            tangent
        )

        # (C) 방향성 보상 (절댓값 사용)
        r_dir = abs(cos_val)

        # (D) brush는 state 업데이트만 담당
        self._apply_brush_vessel((nz, ny, nx))

        reward = r_dir
        done = False

        # 상태 업데이트
        self.pos = (nz, ny, nx)
        self.step_count += 1

        if self.step_count >= self.max_steps:
            done = True

        return self._get_state(), reward, done, False, {
            "pos": self.pos,
            "case": self.current_case_dir.name,
            "reason": "normal"
        }

In [101]:
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")
env = CTVesselCenterEnv(DATA_ROOT)
s, info = env.reset()
print(s.shape, info)

Found 100 cases under /Users/srpark/Projects/RL_project/data/processed/RL_cleaned
(2, 32, 32, 32) {'pos': (np.int64(266), np.int64(270), np.int64(47)), 'case': '01011ug_508'}


In [103]:
from pathlib import Path

# 1) split 디렉토리 / data root
SPLIT_DIR = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned/splits")
DATA_ROOT = Path("/Users/srpark/Projects/RL_project/data/processed/RL_cleaned")

# --------------------------------------------------
# 2) train split ID 읽기
# --------------------------------------------------
train_ids = read_split_ids(SPLIT_DIR, "train")
print(f"🔹 train IDs: {len(train_ids)}개, 예시 5개:", train_ids[:5])

# ID → 실제 case 폴더 경로
train_case_dirs = get_case_dirs(DATA_ROOT, train_ids)
print(f"🔹 train case dirs: {len(train_case_dirs)}개")
for p in train_case_dirs[:3]:
    print("   ", p)

# --------------------------------------------------
# 3) Env를 train set 전용으로 생성
#    (env.case_dirs 를 train 전용으로 교체)
# --------------------------------------------------

def make_train_env(env_class, data_root: Path, train_dirs):
    env = env_class(data_root)
    env.case_dirs = train_dirs  # 🔑 여기서 train set만 사용하게 고정
    print(f"✨ Train-env created! (cases = {len(env.case_dirs)})")
    return env

train_env = make_train_env(CTVesselCenterEnv, DATA_ROOT, train_case_dirs)

# --------------------------------------------------
# 4) sanity check
# --------------------------------------------------
s, info = train_env.reset()
print("state shape:", s.shape)
print("start case:", info["case"], "| start pos:", info["pos"])

🔹 train IDs: 69개, 예시 5개: ['01018ug_73', '01011ug_506', '01015ug_89', '01015ug_100', '01015ug_72']
🔹 train case dirs: 69개
    /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/01018ug_73
    /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/01011ug_506
    /Users/srpark/Projects/RL_project/data/processed/RL_cleaned/01015ug_89
Found 100 cases under /Users/srpark/Projects/RL_project/data/processed/RL_cleaned
✨ Train-env created! (cases = 69)
state shape: (2, 32, 32, 32)
start case: 01015ug_89 | start pos: (np.int64(202), np.int64(398), np.int64(18))


In [ ]:
# ============================================================
# 4. 3D CNN Q-network & ReplayBuffer & train_step
# ============================================================

class CnnQNet(nn.Module):
    def __init__(self, in_channels=2, n_actions=6):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),  # (2,32,32,32) -> (32,16,16,16)

            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),  # -> (64,8,8,8)
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 8 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions)   # Q-values
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        q = self.fc(x)
        return q

    @torch.no_grad()
    def sample_action(self, state, epsilon=0.1):
        """
        state: np array (2,32,32,32)
        """
        if np.random.rand() < epsilon:
            return np.random.randint(0, 6)

        s = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
        q = self.forward(s)
        return int(torch.argmax(q, dim=1).item())


from collections import deque

class ReplayBuffer:
    def __init__(self, capacity=50000):
        self.buffer = deque(maxlen=capacity)

    def push(self, s, a, r, ns, done):
        self.buffer.append((s, a, r, ns, done))

    def sample(self, batch_size=64):
        idx = np.random.choice(len(self.buffer), batch_size, replace=False)
        s, a, r, ns, done = zip(*(self.buffer[i] for i in idx))
        return (
            np.array(s, dtype=np.float32),
            np.array(a, dtype=np.int64),
            np.array(r, dtype=np.float32),
            np.array(ns, dtype=np.float32),
            np.array(done, dtype=np.float32),
        )

    def __len__(self):
        return len(self.buffer)
    



### 학습

In [106]:
import os
import time
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

def train_dqn(
    env,
    num_episodes=3000,
    max_steps_per_ep=500,
    batch_size=64,
    gamma=0.99,
    lr=1e-4,
    epsilon_start=0.9,
    epsilon_end=0.05,
    epsilon_decay=2000,
    ckpt_dir="checkpoints_centerline",
    ckpt_every=100,          # 몇 에피소드마다 weight 저장할지
):
    os.makedirs(ckpt_dir, exist_ok=True)

    q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
    target_net.load_state_dict(q_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(q_net.parameters(), lr=lr)
    buffer = ReplayBuffer(capacity=50000)

    eps = epsilon_start
    global_step = 0

    reward_hist = []
    best_avg10 = -1e9
    best_ckpt_path = os.path.join(ckpt_dir, "best_model.pth")

    print(">>> Training DQN started...")
    t0 = time.time()

    for ep in range(1, num_episodes+1):
        state, info = env.reset()
        total_reward = 0.0

        for step in range(max_steps_per_ep):
            # epsilon decay (step 기반)
            eps = epsilon_end + (epsilon_start - epsilon_end) * \
                  np.exp(-1.0 * global_step / epsilon_decay)

            action = q_net.sample_action(state, eps)

            next_state, reward, done, _, info2 = env.step(action)

            buffer.push(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            global_step += 1

            # --- DQN update ---
            if len(buffer) >= batch_size:
                s, a, r, ns, d = buffer.sample(batch_size)

                s_t  = torch.from_numpy(s).to(DEVICE)
                a_t  = torch.from_numpy(a).to(DEVICE)
                r_t  = torch.from_numpy(r).to(DEVICE)
                ns_t = torch.from_numpy(ns).to(DEVICE)
                d_t  = torch.from_numpy(d).to(DEVICE)

                q = q_net(s_t).gather(1, a_t.unsqueeze(1)).squeeze(1)

                with torch.no_grad():
                    q_next = target_net(ns_t).max(1)[0]
                    target = r_t + gamma * q_next * (1 - d_t)

                loss = F.smooth_l1_loss(q, target)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # 타깃 네트워크는 일정 step마다 동기화
            if global_step % 500 == 0:
                target_net.load_state_dict(q_net.state_dict())

            if done:
                break

        reward_hist.append(total_reward)

        # ---- 로그 출력 ----
        if ep % 10 == 0:
            avg10 = float(np.mean(reward_hist[-10:]))
            elapsed = time.time() - t0
            print(f"[Ep {ep:04d}] "
                  f"reward={total_reward:7.1f} (avg10={avg10:7.1f}), "
                  f"steps={step+1:4d}, eps={eps:0.2f}, "
                  f"buffer={len(buffer):5d}, case={info['case']}, "
                  f"time={elapsed/60:4.1f} min")

            # best model 갱신
            if avg10 > best_avg10:
                best_avg10 = avg10
                torch.save(q_net.state_dict(), best_ckpt_path)
                print(f"   ↳ ✅ best model updated (avg10={best_avg10:.1f})")

        # ---- 에피소드 단위 체크포인트 저장 ----
        if ep % ckpt_every == 0:
            ckpt_path = os.path.join(ckpt_dir, f"ep{ep:04d}.pth")
            torch.save(q_net.state_dict(), ckpt_path)
            print(f"   ↳ 💾 checkpoint saved to {ckpt_path}")

    # 학습 끝난 후 최종 weight 저장
    final_path = os.path.join(ckpt_dir, "final_model.pth")
    torch.save(q_net.state_dict(), final_path)
    print(f"\n>>> Training completed! Final model saved to {final_path}")
    print(f"    Best model (avg10={best_avg10:.1f}) saved to {best_ckpt_path}")

    # ---- reward curve 시각화 & 저장 ----
    plt.figure(figsize=(8,4))
    plt.plot(reward_hist, label="episode reward")
    if len(reward_hist) >= 10:
        # 10-episode moving average
        ma = np.convolve(reward_hist, np.ones(10)/10, mode="valid")
        plt.plot(range(9, 9+len(ma)), ma, label="moving avg (10ep)")
    plt.xlabel("Episode")
    plt.ylabel("Reward")
    plt.title("Training Reward Curve")
    plt.legend()
    plt.grid(True)

    curve_path = os.path.join(ckpt_dir, "reward_curve.png")
    plt.tight_layout()
    plt.savefig(curve_path, dpi=150)
    plt.close()
    print(f"📈 Reward curve saved to {curve_path}")

    # reward history도 나중에 분석용으로 저장
    np.save(os.path.join(ckpt_dir, "reward_hist.npy"), np.array(reward_hist))

    return q_net, target_net, reward_hist

In [108]:
# train_env 는 이미 train split 전용으로 만들어져 있다고 가정
q_net, target_net, reward_hist = train_dqn(
    env=train_env,
    num_episodes=500,
    max_steps_per_ep=1000,
    ckpt_dir="checkpoints_centerline",
    ckpt_every=50,   # 100에피소드마다 ep0100.pth, ep0200.pth ... 저장
)

>>> Training DQN started...
[Ep 0010] reward=   -0.0 (avg10=    3.4), steps=   2, eps=0.86, buffer=  105, case=01010ug_169, time= 1.1 min
   ↳ ✅ best model updated (avg10=3.4)
[Ep 0020] reward=   48.4 (avg10=    8.3), steps=  97, eps=0.78, buffer=  299, case=01011ug_505, time= 4.2 min
   ↳ ✅ best model updated (avg10=8.3)
[Ep 0030] reward=   -0.2 (avg10=    6.5), steps=   3, eps=0.73, buffer=  453, case=01010ug_133, time= 6.7 min
[Ep 0040] reward=   -0.4 (avg10=   19.4), steps=   5, eps=0.61, buffer=  825, case=01018ug_78, time=12.1 min
   ↳ ✅ best model updated (avg10=19.4)
[Ep 0050] reward=  548.5 (avg10=   62.5), steps= 822, eps=0.39, buffer= 1824, case=01011ug_526, time=26.1 min
   ↳ ✅ best model updated (avg10=62.5)
   ↳ 💾 checkpoint saved to checkpoints_centerline/ep0050.pth
[Ep 0060] reward=    0.8 (avg10=    2.5), steps=   5, eps=0.38, buffer= 1903, case=01018ug_94, time=27.5 min
[Ep 0070] reward=    0.1 (avg10=   83.8), steps=   5, eps=0.22, buffer= 3193, case=01018ug_94, time

KeyboardInterrupt: 

In [109]:
torch.save(q_net.state_dict(), "ex05_checkpoint_centerline_latest.pth")

### validation set 확인

In [130]:
# ============================================================
# 4. 3D CNN Q-network & ReplayBuffer & train_step
# ============================================================

class CnnQNet(nn.Module):
    def __init__(self, in_channels=2, n_actions=6):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),  # (2,32,32,32) -> (32,16,16,16)

            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(2),  # -> (64,8,8,8)
        )

        self.fc = nn.Sequential(
            nn.Linear(64 * 8 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, n_actions)   # Q-values
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        q = self.fc(x)
        return q

    @torch.no_grad()
    def sample_action(self, state, epsilon=0.1):
        """
        state: np array (2,32,32,32)
        """
        if np.random.rand() < epsilon:
            return np.random.randint(0, 6)

        s = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
        q = self.forward(s)
        return int(torch.argmax(q, dim=1).item())


from collections import deque

class ReplayBuffer:
    def __init__(self, capacity=50000):
        self.buffer = deque(maxlen=capacity)

    def push(self, s, a, r, ns, done):
        self.buffer.append((s, a, r, ns, done))

    def sample(self, batch_size=64):
        idx = np.random.choice(len(self.buffer), batch_size, replace=False)
        s, a, r, ns, done = zip(*(self.buffer[i] for i in idx))
        return (
            np.array(s, dtype=np.float32),
            np.array(a, dtype=np.int64),
            np.array(r, dtype=np.float32),
            np.array(ns, dtype=np.float32),
            np.array(done, dtype=np.float32),
        )

    def __len__(self):
        return len(self.buffer)
    



In [131]:
q_net = CnnQNet(in_channels=2, n_actions=6).to(DEVICE)
state_dict = torch.load("checkpoints_centerline/ep0250.pth", map_location=DEVICE)
q_net.load_state_dict(state_dict)
q_net.eval()

print("✅ model checkpoint loaded successfully!")

✅ model checkpoint loaded successfully!


In [132]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# =========================================
# 1) val split 읽어서 case 디렉토리 리스트 만들기
# =========================================
SPLIT_DIR = DATA_ROOT / "splits"

val_ids = read_split_ids(SPLIT_DIR, "val")
val_case_dirs = get_case_dirs(DATA_ROOT, val_ids)

print(f"val set 케이스 수: {len(val_case_dirs)}")
print("예시 5개:", [p.name for p in val_case_dirs[:5]])

# =========================================
# 2) 평가용 env 생성 (train env와 동일 + case_dirs만 val로 제한)
# =========================================
env_val = CTVesselCenterEnv(
    data_root=DATA_ROOT,
    patch_size=PATCH_SIZE,
    max_steps=MAX_STEPS,
    brush_radius=BRUSH_RADIUS,
)

# train 때는 전체 case가 들어가 있었을 텐데,
# 여기서는 val 케이스만 쓰도록 덮어쓰기
env_val.case_dirs = val_case_dirs

# =========================================
# 3) greedy rollout + info
# =========================================
def rollout_greedy_with_info(env, q_net, max_steps=200):
    state, info = env.reset()
    traj = [tuple(info["pos"])]
    done = False
    last_info = info
    steps = 0

    while (not done) and steps < max_steps:
        s_t = torch.from_numpy(state).float().unsqueeze(0).to(DEVICE)
        q_values = q_net(s_t)         # (1, 6)
        action = q_values.argmax(dim=1).item()  # ← greedy 선택

        state, reward, done, _, info2 = env.step(action)
        traj.append(tuple(info2["pos"]))
        last_info = info2
        steps += 1

    return np.array(traj), info, last_info, steps

# =========================================
# 4) centerline + trajectory 3D plot & 저장
# =========================================
def plot_centerline_and_traj(env, traj, case_name,
                             local_only=True, pad=40,
                             save_path=None):
    """
    env.centerline 과 주어진 traj를 함께 3D로 그림.
    local_only=True 이면 trajectory 근처만 잘라서 그림.
    """
    cl_vol = env.centerline
    cl_idx = np.argwhere(cl_vol > 0)   # (Nc, 3) : (z,y,x)

    zt, yt, xt = traj[:, 0], traj[:, 1], traj[:, 2]

    if local_only:
        z_min, z_max = max(0, zt.min() - pad), min(cl_vol.shape[0], zt.max() + pad)
        y_min, y_max = max(0, yt.min() - pad), min(cl_vol.shape[1], yt.max() + pad)
        x_min, x_max = max(0, xt.min() - pad), min(cl_vol.shape[2], xt.max() + pad)

        sel = (
            (cl_idx[:, 0] >= z_min) & (cl_idx[:, 0] <= z_max) &
            (cl_idx[:, 1] >= y_min) & (cl_idx[:, 1] <= y_max) &
            (cl_idx[:, 2] >= x_min) & (cl_idx[:, 2] <= x_max)
        )
        cl_plot = cl_idx[sel]
    else:
        cl_plot = cl_idx

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection="3d")

    # centerline
    if len(cl_plot) > 0:
        ax.scatter(
            cl_plot[:, 2], cl_plot[:, 1], cl_plot[:, 0],
            s=1, alpha=0.05, label="centerline"
        )

    # trajectory
    ax.plot(traj[:, 2], traj[:, 1], traj[:, 0],
            linewidth=3, label="trajectory")

    # start / end
    ax.scatter(traj[0, 2], traj[0, 1], traj[0, 0],
               s=40, c="green", label="start")
    ax.scatter(traj[-1, 2], traj[-1, 1], traj[-1, 0],
               s=40, c="red", label="end")

    ax.set_xlabel("x (col)")
    ax.set_ylabel("y (row)")
    ax.set_zlabel("z (slice)")
    ax.set_title(f"3D trajectory on centerline | case {case_name}")
    ax.legend(loc="upper left")
    ax.view_init(elev=25, azim=-60)
    plt.tight_layout()

    if save_path is not None:
        fig.savefig(save_path, dpi=150)
        plt.close(fig)
    else:
        plt.show()

# =========================================
# 5) 전체 val set에 대해 inference + PNG 저장
# =========================================
save_dir = "val_traj_ep0250"
os.makedirs(save_dir, exist_ok=True)

results = []

for cdir in val_case_dirs:
    # 이 케이스만 사용하도록 env 내부 case_dirs를 1개로 잠시 제한
    env_val.case_dirs = [cdir]

    traj, info0, last_info, steps = rollout_greedy_with_info(
        env_val, q_net, max_steps=200
    )
    case_name = cdir.name
    reason = last_info.get("reason", "unknown")

    save_path = os.path.join(save_dir, f"traj_{case_name}.png")
    plot_centerline_and_traj(
        env_val, traj, case_name,
        local_only=True, pad=40,
        save_path=save_path
    )

    results.append((case_name, steps, reason))
    print(f"[{case_name}] steps={steps:4d}, end_reason={reason}")

print("\n✅ val set inference 완료!")
print(f"PNG 파일 위치: {save_dir}/traj_*.png")

val set 케이스 수: 14
예시 5개: ['01018ug_86', '01018ug_72', '01018ug_87', '01010ug_8', '01018ug_85']
Found 100 cases under /Users/srpark/Projects/RL_project/data/processed/RL_cleaned
[01018ug_86] steps= 200, end_reason=normal
[01018ug_72] steps= 200, end_reason=normal
[01018ug_87] steps= 200, end_reason=normal
[01010ug_8] steps= 200, end_reason=normal
[01018ug_85] steps=   2, end_reason=out_of_vessel
[01015ug_92] steps= 200, end_reason=normal
[01015ug_97] steps= 200, end_reason=normal
[01010ug_134] steps=   4, end_reason=out_of_vessel
[01010ug_164] steps= 200, end_reason=normal
[01011ug_529] steps= 200, end_reason=normal
[01018ug_81] steps= 200, end_reason=normal
[01018ug_84] steps= 200, end_reason=normal
[01011ug_502] steps= 200, end_reason=normal
[01010ug_174] steps=   1, end_reason=out_of_vessel

✅ val set inference 완료!
PNG 파일 위치: val_traj_ep0250/traj_*.png


## 05. Reward 수정

- Centerline cosine similarity + distance 가중치